# Computational Social Science - Seminar Project
### Lexical Comparison Between News Mediums by Using Word Embeddings for Media Bias Identification
## I Data exploration, pre-processing, word embeddings training
Lada Rudnitckaia






## Directories and packages


In [0]:
import os
# for jupyter notebook
# os.chdir('/Users/ladarudnitckaia/Desktop/CSS/Project/queries from ccnc')

# for colaboratory gpu

from google.colab import drive
drive.mount('/content/gdrive')
import sys

os.chdir("/content/gdrive/My Drive/CSS Project")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# !pip install demoji

import demoji
import ast

import pandas as pd
import csv
import time
import re
from random import sample
import statistics

import string
import inflect
infl = inflect.engine()

from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('words')

import warnings  
warnings.filterwarnings(action = 'ignore')

import gensim 
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim import utils
from gensim.corpora import Dictionary
from gensim.utils import tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models.phrases import original_scorer

from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

# from pycontractions import Contractions
# import language_check

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


## 1 HuffPost

### 1.1 Data Exploration

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Data")
left_news = pd.read_csv("left_news_dates.csv")
huffpost = left_news[left_news.source_domain == "www.huffingtonpost.com"]
del(left_news)
huffpost.head()

,date_publish,lang,source_domain,title,text
2384,2018-07-23 13:44:16,en,www.huffingtonpost.com,"Yes, Democrats Need To Run Left - On Economics",Spencer Platt via Getty Images Rising progress...
2385,2017-09-27 22:45:42,en,www.huffingtonpost.com,Trump's Explanation For Removing Sudan From Hi...,"At a press briefing on Wednesday afternoon, U...."
2386,2017-10-02 17:56:46,en,www.huffingtonpost.com,It's Completely Legal To Walk Around Las Vegas...,Sunday’s horrific mass shooting in Las Vegas o...
2387,2017-08-01 14:55:48,en,www.huffingtonpost.com,Dylan Sprouse Is Acting Again For The First Ti...,Dylan Sprouse is returning to screens after ta...
2388,2016-12-30 16:47:06,en,www.huffingtonpost.com,How to Tell if You've Really Found Your Soulmate,How can you be sure the person you're going to...


In [0]:
print('Number of articles:', huffpost.shape[0]) # 101108

Number of articles: 101108


#### 1.1.1 Select only articles published in or later than 2010

In [0]:
huffpost['year_publish'] = huffpost.apply(lambda row: int(row.date_publish[:4]) if (pd.notnull(row.date_publish)) else row.date_publish, axis=1)

In [0]:
print("The number of articles published in or later than 2010:",
      len(huffpost[huffpost.year_publish >= 2010]), ",",
      round(len(huffpost[huffpost.year_publish >= 2010])/len(huffpost[pd.notnull(huffpost.year_publish)])*100,2),
      "%")
print("The number of articles published earlier than 2010:",
      len(huffpost[huffpost.year_publish < 2010]), ",",
      round(len(huffpost[huffpost.year_publish < 2010])/len(huffpost[pd.notnull(huffpost.year_publish)])*100,2),
      "%")

The number of articles published in or later than 2010: 101101 , 100.0 %
The number of articles published earlier than 2010: 1 , 0.0 %


In [0]:
print("The number of articles with missing publish date:", huffpost.date_publish.isnull().sum())

The number of articles with missing publish date: 6


Articles with unknown publish date are kept due to the scarcity of the data. Basing on the rest of the data, I assume that majority of them were published later than 2010.

In [0]:
huffpost = pd.concat([huffpost[huffpost.year_publish >= 2010],
                        huffpost[huffpost.year_publish.isnull() == True]])

#### 1.1.2 Missing texts

In [0]:
print('Is there missing values?', huffpost.isnull().values.any())
print('Number of missing values:\n', huffpost.isnull().sum())
# Delete the rows with missing article text:
huffpost = huffpost.dropna(subset=['text'])

Is there missing values? True
Number of missing values:
 date_publish      6
lang              0
source_domain     0
title             0
text             10
year_publish      6
dtype: int64


In [0]:
print('Number of articles after filtering:', huffpost.shape[0]) # 101097

Number of articles after filtering: 101097


### 1.2 Analysis of pre-processing options 

#### 1.2.1 Include title in the text

In [0]:
huffpost['text_preprocessed'] = huffpost["title"].astype(str) + ". " + huffpost["text"]

#### 1.2.2 Remove links and nicknames

In [0]:
http_number, www_number, pic_number, at_number = 0, 0, 0, 0
for row in huffpost.text_preprocessed:
  if "http" in row:
    http_number += 1
  if 'www' in row:
    www_number += 1
  if 'pic.' in row:
    pic_number += 1
  if '@' in row:
    at_number += 1

print('The number of articles containing "http" in the text', http_number)
print('The number of articles containing "www" in the text', www_number)
print('The number of articles containing "pic." in the text', pic_number)
print('The number of articles containing "@" in the text', at_number)

The number of articles containing "http" in the text 7075
The number of articles containing "www" in the text 2870
The number of articles containing "pic." in the text 9497
The number of articles containing "@" in the text 18196


In [0]:
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'http\S+', '', row.text_preprocessed), axis=1) # including https
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'www\.\S+', '', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'pic\.\S+', '', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'@\S+', '', row.text_preprocessed), axis=1)

#### 1.2.3 Gensim simple pre-processing

In [0]:
start_time = time.time()
huffpost['tokens'] = huffpost.apply(lambda row: gensim.utils.simple_preprocess(row.text_preprocessed, 
                                                                               deacc=False, min_len=1, max_len=28),
                                    axis=1)
end_time = time.time()
print("huffpost gensim simple_preprocess", round((end_time - start_time),2), "seconds")

huffpost gensim simple_preprocess 78.97 seconds


In [0]:
huffpost['words'] = huffpost.apply(lambda row: len(row.tokens), axis=1)

print("Total number of words for training:", sum(huffpost['words']))
print("Average number of words per article:", round(statistics.mean(huffpost['words']),0))

Total number of words for training: 68172239
Average number of words per article: 674.0


#### 1.2.4 Manual pre-processing

##### 1.2.4.1 Lowercase

In [0]:
huffpost['text_preprocessed'] = huffpost.apply(lambda row: row.text_preprocessed.lower(), axis=1)

##### 1.2.4.2 Remove emoji

In [0]:
demoji.download_codes()

... OK (Got response in 0.10 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [0]:
start_time = time.time()
huffpost['text_preprocessed'] = huffpost.apply(lambda row: demoji.replace(row.text_preprocessed, repl=''), axis=1)
end_time = time.time()

print('Time to remove emoji:', round((end_time - start_time),2), 'seconds')

Time to remove emoji: 1255.05 seconds


##### 1.2.4.3 Remove particular dashs or hyphens

I want to keep the compound words like "anti-women" complete. The hyphens, however, can be also used as a replacement for a comma, when the subsequent clause significantly shifts the primary focus of the preceding text, e.g., “They are afraid of no one—except apparently their own passengers especially those who are Black.” In this case the words "one" and "except" should indeed be split.

There are different types of dashs and hyphens: The dash ( ‒, –, —, ― ) and hyphen or hyphen-minus ( ‐ ) [Wikipedia]. There is also the keyboard dash ( - ). And, as the post pre-processing analysis of the words with length equal to 1 shows, there are some more!

P.s. In the code in Collaboratory, they look similar but in fact Python recognizes them and the keybord dash as different marks. Therefore, I will sample the text containing such marks to check when they are used and whether they should be removed or not.

In [0]:
hyphens = {'small_dash':'‒', 'bigger_dash':'–', 'big_dash':'—', 'thebiggest_dash':'―', 'hyp_minus':'‐', 'keybord':'-'}

comparison = {}
for key, value in hyphens.items():
  comp = []
  for key_rest, value_rest in hyphens.items():
    comp.append(key_rest + ' ' + str(value == value_rest))
  comparison[key] = comp

comparison # they are indeed all different

In [0]:
small_dash_articles, small_dash_number = [], 0
bigger_dash_articles, bigger_dash_number = [], 0
big_dash_articles, big_dash_number = [], 0
thebiggest_dash_articles, thebiggest_dash_number = [], 0
hyp_minus_dash_articles, hyp_minus_dash_number = [], 0
keybord_dash_articles, keybord_dash_number = [], 0
# new, discovered after analysis of len=1 >>
another_dash_articles, another_dash_number = [], 0
one_more_dash_articles, one_more_dash_number = [], 0
again_articles, again_number = [], 0
and_again_articles, and_again_number = [], 0
moreee_articles, moreee_number = [], 0
for row in huffpost.text:
  if '‒' in row:
    small_dash_number += 1
    small_dash_articles.append(row)
  if '–' in row: 
    bigger_dash_number += 1
    bigger_dash_articles.append(row)
  if '—' in row:
    big_dash_number += 1
    big_dash_articles.append(row)
  if '―' in row:
    thebiggest_dash_number += 1
    thebiggest_dash_articles.append(row)
  if '‐' in row:
    hyp_minus_dash_number += 1
    hyp_minus_dash_articles.append(row)
  if '-' in row:
    keybord_dash_number += 1
    keybord_dash_articles.append(row)
  # new, discovered after analysis of len=1 >>
  if '⎯' in row:
    another_dash_number += 1
    another_dash_articles.append(row)
  if '⏤' in row:
    one_more_dash_number += 1
    one_more_dash_articles.append(row)
  if '─' in row:
    again_number += 1
    again_articles.append(row)
  if '－' in row:
    and_again_number += 1
    and_again_articles.append(row)
  if '−' in row:
    moreee_number += 1
    moreee_articles.append(row)

print('The number of articles containing small_dash in the text', small_dash_number)
print('The number of articles containing bigger_dash in the text', bigger_dash_number)
print('The number of articles containing big_dash in the text', big_dash_number)
print('The number of articles containing thebiggest_dash in the text', thebiggest_dash_number)
print('The number of articles containing hyp_minus in the text', hyp_minus_dash_number)
print('The number of articles containing keybord in the text', keybord_dash_number)
# new, discovered after analysis of len=1 >>
print('The number of articles containing another_dash in the text', another_dash_number)
print('The number of articles containing one_more_dash in the text', one_more_dash_number)
print('The number of articles containing again in the text', again_number)
print('The number of articles containing and_again in the text', and_again_number)
print('The number of articles containing moreee in the text', moreee_number)

The number of articles containing small_dash in the text 7
The number of articles containing bigger_dash in the text 13175
The number of articles containing big_dash in the text 32771
The number of articles containing thebiggest_dash in the text 23350
The number of articles containing hyp_minus in the text 27
The number of articles containing keybord in the text 85283
The number of articles containing another_dash in the text 2
The number of articles containing one_more_dash in the text 4
The number of articles containing again in the text 17
The number of articles containing and_again in the text 1
The number of articles containing moreee in the text 19


For each type of dash / hyphen, approximately 200-250 cases are manually examined:

small dash ( ‒ ): TO REMOVE. Examples: 
*   "... what they need to do ‒ in fact, they do ..."
*   Also connects time periods, e.g., 1950‒1960.
*   Doesn't connect compound words!

bigger dash ( – ): TO REMOVE. Examples: 
*   "... reacted differently to a strong dollar–and the notion of ..."
*   Doesn't connect compound words!
*   Some will be removed automatically, e.g., "what they need to do – in fact, they do".
*   Also connects ranges, e.g., 1950–1960 or $40–70/bbl

big dash ( — ): TO REMOVE. Examples:
*   "The next one—due out this month—won’t be released."
*   Doesn't connect compound words!
*   Some will be removed automatically, e.g., "Becca tells Arie to leave — repeatedly — and he stays".

the biggest dash ( ― ): TO REMOVE. Examples:
*   "...determined to expunge it―never mind whether the policy is ..."
*   Will be removed automatically e.g., "Trump’s tweets ― which appeared ..."

hyphen minus dash ( ‐ ): TO KEEP. Example:
*   connects compound words! E.g., "self‐regulation"

keyboard dash ( - ): TO KEEP. Example:
*   connects compound words! E.g., "anti-gay"
*   also: "We’ve all been there - not being able to ...", therefore, TO REMOVE ' - ', '- ', ' -'.

In [0]:
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'‒', ' ', row.text_preprocessed), axis=1) # small dash: '‒'
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'–', ' ', row.text_preprocessed), axis=1) # bigger dash: '–'
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'—', ' ', row.text_preprocessed), axis=1) # big dash: '—'
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'―', ' ', row.text_preprocessed), axis=1) # the biggest dash: '―'
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' - ', ' ', row.text_preprocessed), axis=1) # keybord dash: '-'
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'- ', ' ', row.text_preprocessed), axis=1) # keybord dash: '-'
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' -', ' ', row.text_preprocessed), axis=1) # keybord dash: '-'

In [0]:
small_dash_number = 0
bigger_dash_number = 0
big_dash_number = 0
thebiggest_dash_number = 0
hyp_minus_dash_number = 0
keybord_dash_number = 0
for row in huffpost.text_preprocessed:
  if '‒' in row: 
    small_dash_number += 1
  if '–' in row: 
    bigger_dash_number += 1
  if '—' in row:
    big_dash_number += 1
  if '―' in row:
    thebiggest_dash_number += 1
  if '‐' in row:
    hyp_minus_dash_number += 1
  if '-' in row:
    keybord_dash_number += 1

print('The number of articles still containing small_dash in the text', small_dash_number)
print('The number of articles still containing bigger_dash in the text', bigger_dash_number)
print('The number of articles still containing big_dash in the text', big_dash_number)
print('The number of articles still containing thebiggest_dash in the text', thebiggest_dash_number)
print('The number of articles still containing hyp_minus_dash in the text', hyp_minus_dash_number)
print('The number of articles still containing keybord_dash in the text', keybord_dash_number)

The number of articles still containing small_dash in the text 0
The number of articles still containing bigger_dash in the text 0
The number of articles still containing big_dash in the text 0
The number of articles still containing thebiggest_dash in the text 0
The number of articles still containing hyp_minus_dash in the text 0
The number of articles still containing keybord_dash in the text 40


##### 1.2.4.4 Handle contractions and possessions

Only unambigous contractions are expanded with regex and manually created disctionary from Wikipedia. First, Wikipedia says that many more contractions exist. Second, ambigous contractions could be removed with pycontractions package that provides the grammar check and expands correctly. However, with my PC, it would take approximatey 13 days for 100,000 articles. Therefore, I will keep the contractions as they are.

There 2 types of punctuation marks: inverted comma ’ and apostrophe '.

I will use the package "string" for punctuation removal (section 1.2.7). Apostrophes ' or inverted commas ’ will be removed from the punctuation marks list for subsequent removal. I also removed the keybord dash from the list since it connects compound words and I want to keep them with the dash. All other dashs are handled earlier.
The inverted commas can also be used as quotation marks ‘...’, therefore, I will remove the closing quotation mark, assuming that when inverted comma is used for as closing quotation mark, it is followed by the space '’ '. Some are, however, followed by other punctuation marks. Therefore, I will remove '’ ' again after all punctuation marks removal. The text also contains such weird compositions: ‘’transforming urban life’’. I will also manually remove them.

In [0]:
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'’ ', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' ’ ', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' ’', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r"' ", ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r" '", ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r" ' ", ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'‘’', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'’’', ' ', row.text_preprocessed), axis=1)

##### 1.2.4.5 Remove punctuation

Another punctuation marks from Wikipedia are added: 

⟨⟩ … “ ” « » ‘ ⁄

Plus, the marks, that were discovered after post pre-processing analysis of the words with the length equal to 1:

≤ ≥ ≠ ॥ × ÷ ″ ± † ✝ § € £ ° ≤ ⎯ ⏤ − ─ ® © ™ ½ ¾ ⅔ ¼ ⅛ ⅓ ■ ○ ● • · ｡ ☥ ✡

In [0]:
# string package list of punctuation marks for subsequent removal
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
punctuation_removal_string = '!"#$%&\()*+,./:;<=>≤≥≠॥×÷?@[\\]^_`{|}~⟨⟩…“”″«»‘⁄±†✝§€£°≤⎯⏤−─®©™½¾⅔¼⅛⅓■○●•·｡☥✡' 

In [0]:
huffpost['text_preprocessed'] = huffpost.apply(lambda row: row.text_preprocessed.translate(str.maketrans('', '', punctuation_removal_string)), axis=1)

I will one more time remove the apostophes ' and inverted commas ’ that could be used as quotation marks.

After analysis of the words with length = 1, I found around 700 keyboard dashs '-'. I one more time remove them, except those that form the compound words, therefore I will only remove those that are surrounded by spaces.

In [0]:
# apostophes ' and inverted commas ’
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'’ ', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' ’ ', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' ’', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r"' ", ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r" '", ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r" ' ", ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'‘’', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'’’', ' ', row.text_preprocessed), axis=1)
# keyboard dashs -
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'- ', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' - ', ' ', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r' -', ' ', row.text_preprocessed), axis=1)

##### 1.2.4.6 Tokenization to words

NLTK and Gensim tokenizer split words that contain punctuation marks inside (compound words, contractions, possessions), e.g., anti-gay -> 'anti', 'gay'; Jack’s -> 'Jack', ' ’ ', 's'.

Gensim also automatically removes all punctuation marks and numbers.

Therefore, I will just use the simple string .split()

In [0]:
huffpost['text_preprocessed'] = huffpost.apply(lambda row: row.text_preprocessed.split(), axis=1)

##### 1.2.4.7 Replace numbers with respective words

In [0]:
start_time = time.time()
for (index, row) in huffpost.iterrows():
  for index, token in enumerate(row.text_preprocessed):
    try:
      row.text_preprocessed[index] = infl.number_to_words(int(token))
    except:
      continue
end_time = time.time()

print('Time to convert numbers to respective words:', round((end_time - start_time),2), 'seconds')

Time to convert numbers to respective words: 159.16 seconds


##### 1.2.4.8 Convert inverted commas to apostrophes and hyp_minus_dash to keybord_dash (for consistency)

In [0]:
start_time = time.time()
for (index, row) in huffpost.iterrows():
  for index, token in enumerate(row.text_preprocessed):
    if "’" in token:
      row.text_preprocessed[index] = re.sub(r"’", "'", token)
    if "‐" in token:
      row.text_preprocessed[index] = re.sub(r"‐", "-", token)
end_time = time.time()

print('Time:', round((end_time - start_time),2), 'seconds')

Time: 51.42 seconds


#### 1.2.5 Pre-processing results analysis

##### 1.2.5.1 Manual sample check: comparison to initial text and to gensim simple pre-processing

In [0]:
sample_for_check = huffpost.sample(n=5, random_state=1234)

In [0]:
for row in sample_for_check.text:
  print(row)

Pope Francis acknowledged that he committed “serious errors” in handling a sex abuse scandal that has roiled Chile’s Roman Catholic Church and resulted in one of the most heated controversies of his papacy. Francis made the admission in a letter sent to Chilean bishops that was published online Wednesday. Francis wrote that hearing about the “crucified lives” of the victims has caused him “pain and shame.” “I recognize and I want you to communicate this accurately, that I have made serious errors of judgment and perception of the situation, especially due to lack of truthful and balanced information,” Francis told the bishops in the letter dated April 8, according to a translation by the Catholic news site Crux. In the coming weeks, Francis will also meet with Chilean victims of clergy sexual abuse and ask each one personally for forgiveness, according to Jaime Coiro, a spokesman for the Chilean bishops’ conference. EITAN ABRAMOVICH via Getty Images People demonstrate against Pope Fran

In [0]:
for row in sample_for_check.text_preprocessed:
  print(row)

['pope', 'admits', 'he', 'made', 'serious', 'errors', 'in', 'handling', 'chile', 'sex', 'abuse', 'allegations', 'pope', 'francis', 'acknowledged', 'that', 'he', 'committed', 'serious', 'errors', 'in', 'handling', 'a', 'sex', 'abuse', 'scandal', 'that', 'has', 'roiled', 'chile’s', 'roman', 'catholic', 'church', 'and', 'resulted', 'in', 'one', 'of', 'the', 'most', 'heated', 'controversies', 'of', 'his', 'papacy', 'francis', 'made', 'the', 'admission', 'in', 'a', 'letter', 'sent', 'to', 'chilean', 'bishops', 'that', 'was', 'published', 'online', 'wednesday', 'francis', 'wrote', 'that', 'hearing', 'about', 'the', 'crucified', 'lives', 'of', 'the', 'victims', 'has', 'caused', 'him', 'pain', 'and', 'shame', 'i', 'recognize', 'and', 'i', 'want', 'you', 'to', 'communicate', 'this', 'accurately', 'that', 'i', 'have', 'made', 'serious', 'errors', 'of', 'judgment', 'and', 'perception', 'of', 'the', 'situation', 'especially', 'due', 'to', 'lack', 'of', 'truthful', 'and', 'balanced', 'information',

In [0]:
sample_for_check['gensim_prep'] = sample_for_check.apply(lambda row: gensim.utils.simple_preprocess(row.text, deacc=False, min_len=1, max_len=100), axis=1)

for row in sample_for_check.gensim_prep:
  print(row)

['pope', 'francis', 'acknowledged', 'that', 'he', 'committed', 'serious', 'errors', 'in', 'handling', 'a', 'sex', 'abuse', 'scandal', 'that', 'has', 'roiled', 'chile', 's', 'roman', 'catholic', 'church', 'and', 'resulted', 'in', 'one', 'of', 'the', 'most', 'heated', 'controversies', 'of', 'his', 'papacy', 'francis', 'made', 'the', 'admission', 'in', 'a', 'letter', 'sent', 'to', 'chilean', 'bishops', 'that', 'was', 'published', 'online', 'wednesday', 'francis', 'wrote', 'that', 'hearing', 'about', 'the', 'crucified', 'lives', 'of', 'the', 'victims', 'has', 'caused', 'him', 'pain', 'and', 'shame', 'i', 'recognize', 'and', 'i', 'want', 'you', 'to', 'communicate', 'this', 'accurately', 'that', 'i', 'have', 'made', 'serious', 'errors', 'of', 'judgment', 'and', 'perception', 'of', 'the', 'situation', 'especially', 'due', 'to', 'lack', 'of', 'truthful', 'and', 'balanced', 'information', 'francis', 'told', 'the', 'bishops', 'in', 'the', 'letter', 'dated', 'april', 'according', 'to', 'a', 'tran

##### 1.2.5.2 Check all tokens with length = 1, were all punctuation marks removed?

###### 1.2.5.2.1 Gensim simple pre-processing

In [0]:
len_one = {}
for (index, row) in huffpost.iterrows():
  for token in row.tokens:
    if len(token) == 1:
      if token not in len_one.keys():
        len_one[token] = 1
      else:
        len_one[token] += 1
        
sorted_len_one = sorted(len_one.items() , reverse=True, key=lambda x: x[1])
for elem in sorted_len_one :
    print(elem[0] , " ::" , elem[1] )

a  :: 1577896
s  :: 787321
i  :: 675489
t  :: 277960
m  :: 61425
u  :: 53608
d  :: 45164
r  :: 14158
c  :: 10942
e  :: 9582
j  :: 8181
b  :: 7907
o  :: 7500
k  :: 7264
n  :: 5770
p  :: 5357
w  :: 4385
g  :: 4075
l  :: 3879
x  :: 3861
y  :: 3157
f  :: 3044
v  :: 2957
h  :: 2918
q  :: 2620
z  :: 1598
à  :: 387
½  :: 385
¼  :: 106
ѕ  :: 54
¾  :: 49
ø  :: 46
º  :: 39
â  :: 37
α  :: 25
è  :: 22
ي  :: 20
¹  :: 18
²  :: 17
³  :: 15
ਕ  :: 15
ن  :: 14
ü  :: 14
ਨ  :: 14
⁴  :: 13
ツ  :: 13
⁵  :: 12
ل  :: 12
ל  :: 11
ਰ  :: 11
ਮ  :: 11
⁶  :: 10
ਤ  :: 10
в  :: 10
ه  :: 10
א  :: 9
ה  :: 9
ਈ  :: 9
ਸ  :: 9
ਹ  :: 9
ਜ  :: 9
í  :: 9
ب  :: 9
و  :: 8
ㅤ  :: 8
⁷  :: 8
⁸  :: 8
и  :: 8
β  :: 7
⅔  :: 7
⅓  :: 7
ñ  :: 7
أ  :: 7
ت  :: 7
á  :: 6
र  :: 6
म  :: 6
ا  :: 6
ਲ  :: 6
儒  :: 6
ك  :: 6
ج  :: 6
د  :: 6
ف  :: 6
ح  :: 6
ਦ  :: 5
ر  :: 5
س  :: 5
م  :: 5
כ  :: 4
ע  :: 4
ך  :: 4
ひ  :: 4
仁  :: 4
ਪ  :: 4
ਇ  :: 4
ਭ  :: 4
а  :: 4
ω  :: 4
י  :: 4
⁹  :: 4
ع  :: 4
ß  :: 3
⅛  :: 3
ם  :: 3
न  :: 3
ਗ  :: 3
я  :: 3
ͳ  :: 3
מ  :

In [0]:
print("Number of tokens with length 1:", len(len_one))

Number of tokens with length 1: 239


###### 1.2.5.2.2 Manual pre-processing

In [0]:
len_one = {}
for (index, row) in huffpost.iterrows():
  for token in row.text_preprocessed:
    if len(token) == 1:
      if token not in len_one.keys():
        len_one[token] = 1
      else:
        len_one[token] += 1
        
sorted_len_one = sorted(len_one.items() , reverse=True, key=lambda x: x[1])
for elem in sorted_len_one :
    print(elem[0] , " ::" , elem[1] )

a  :: 1566802
i  :: 575534
j  :: 5463
b  :: 3576
r  :: 3233
d  :: 3117
w  :: 2943
e  :: 2748
c  :: 2402
v  :: 2213
x  :: 2145
s  :: 1778
m  :: 1707
l  :: 1523
q  :: 1498
f  :: 1343
n  :: 1173
t  :: 1145
u  :: 1077
k  :: 1043
p  :: 987
g  :: 964
o  :: 918
h  :: 852
z  :: 732
y  :: 701
à  :: 273
️  :: 93
​  :: 85
-  :: 58
→  :: 55
ø  :: 46
⁦  :: 38
­  :: 37
♡  :: 30
➞  :: 27
★  :: 24
◦  :: 24
  :: 19
è  :: 18
✪  :: 18
✓  :: 16
‏  :: 16
ü  :: 14
›  :: 14
◡  :: 13
̶  :: 11
¹  :: 10
в  :: 10
⁴  :: 9
ن  :: 9
❏  :: 9
²  :: 8
  :: 8
и  :: 8
³  :: 7
¿  :: 7
⁵  :: 7
ً  :: 7
⁠  :: 7
⁂  :: 7
ñ  :: 7
á  :: 6
◇  :: 6
‎  :: 6
‍  :: 6
☭  :: 6
ਨ  :: 6
➧  :: 6
❦  :: 6
ⓥ  :: 5
′  :: 5
⌐  :: 5
ㅤ  :: 5
↓  :: 5
  :: 5
⁷  :: 5
儒  :: 5
،  :: 5
⦁  :: 5
△  :: 5
☞  :: 5
ひ  :: 4
⁶  :: 4
⁸  :: 4
︎  :: 4
☆  :: 4
а  :: 4
ω  :: 4
☾  :: 4
❖  :: 4
▲  :: 4
α  :: 4
â  :: 3
  :: 3
❀  :: 3
‑  :: 3
ͳ  :: 3
  :: 3
仁  :: 3
�  :: 3
禮  :: 3
̴  :: 3
🕇  :: 3
é  :: 3
  :: 3
◊  :: 3
►  :: 2
✰  :: 2
❁  :: 2
⅗  :: 2
❝  :: 2
♕  

In [0]:
len(len_one)

##### 1.2.5.3 Save manually pre-processed data (long to train)

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Data")
huffpost.to_csv(r'huffington_preprocessed.csv', index = False, header = True)

##### 1.2.5.4 Pre-processing results discussion

GENSIM SIMPLE PREPROCESSING

*+++* 
* All the punctuation marks are removed
* Emojis are removed
* Fast

*---*
* Numbers are removed
* Compound words, contractions and posessions are separated



---


MANUAL PRE-PROCESSING

*+++* 
*   Emojis are removed
*   Numbers are converted to written words
*   Most punctuation marks are removed
*   Compound words, contractions and posessions are kept complete, with hyphens / apostrophes

*---*
*   Some abbreviations that are kept complete, e.g. "U.S." is now equal to the pronoun "us"
*   There is still noise
*   There still mistakes
* Can work differently for other news outlet
* Slow
* Decisions on pre-processing steps are based on manual analysis of small number of articles



---


Compare to wordlist

Compare the noise when creating bigrams

### 1.3 Analysis of word embeddings trained on differently pre-processed data

#### 1.3.1 Gensim simple pre-processing, no bigrams

##### 1.3.1.1 Training

In [0]:
start_time = time.time()
huff_simp_nobigr = gensim.models.Word2Vec(huffpost['tokens'],
                                 size = 300, window = 8, min_count = 25, workers = 4, sample = 0.00001,
                                 sg = 1, compute_loss = True, callbacks = ())
end_time = time.time()

os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_simp_nobigr.save("huff_simp_nobigr.model")
print("Word2Vec for left news (Huffington Post) for", round((end_time - start_time),2), "seconds")

Word2Vec for left news (Huffington Post) for 619.44 seconds


##### 1.3.1.2 Analysis of quality

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_simp_nobigr = Word2Vec.load("huff_simp_nobigr.model")

###### 1.3.2.3.1 Vocabulary

In [0]:
print("Vocabulary length:", len(huff_simp_nobigr.wv.vocab), "words")

Vocabulary length: 43913 words


In [0]:
huff_simp_nobigr_vocab = {}
for token in huff_simp_nobigr.wv.vocab:
  huff_simp_nobigr_vocab[token] = huff_simp_nobigr.wv.vocab[token].count

huff_simp_nobigr_vocab = pd.DataFrame.from_dict(huff_simp_nobigr_vocab, orient='index', columns=['frequency'])
huff_simp_nobigr_vocab['token'] = huff_simp_nobigr_vocab.index
huff_simp_nobigr_vocab = huff_simp_nobigr_vocab.sort_values(by='frequency', ascending=False)
huff_simp_nobigr_vocab = huff_simp_nobigr_vocab.reset_index(inplace = False) 
huff_simp_nobigr_vocab = huff_simp_nobigr_vocab[['token', 'frequency']]

huff_simp_nobigr_vocab.head()

,token,frequency
0,the,3347564
1,to,2016132
2,and,1854588
3,of,1656445
4,a,1577896


In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")
huff_simp_nobigr_vocab.to_csv(r'huff_simp_nobigr_VOCAB.csv', index = False, header = True)

###### 1.3.1.2.1 Word semantic similarity

Word semantic similarity will be tested on four following datasets:

 - **WordSim-353**, 353 pairs assessed by semantic similarity (however, some researchers find the instructions for assessors ambiguous with respect to similarity and association) with a scale from 0 to 10,
 - **SimVerb-3500**, 3,500 pairs of verbs assessed by semantic similarity (that means that pairs that are related but not similar have a fairly low rating) with a scale from 0 to 4,
 - **MEN**, 3,000 pairs assessed by semantic relatedness with a discrete scale from 0 to 50,
 - **RW**, 2,034 pairs of words with low occurrences (rare words) assessed by semantic similarity with a scale from 0 to 10.

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")

In [0]:
wordsim353 = huff_simp_nobigr.evaluate_word_pairs('wordsim353.tsv')
simverb3500 = huff_simp_nobigr.evaluate_word_pairs('simverb-3500.txt')
men = huff_simp_nobigr.evaluate_word_pairs('men.txt')
rw = huff_simp_nobigr.evaluate_word_pairs('rw.txt')

print("Test dataset WordSim-353 Pearson:", round(wordsim353[0][0],2), ", Spearman:", round(wordsim353[1][0],2))
print("Test dataset SimVerb-3500 Pearson:", round(simverb3500[0][0],2), ", Spearman:", round(simverb3500[1][0],2))
print("Test dataset MEN Pearson:", round(men[0][0],2), ", Spearman:", round(men[1][0],2))
print("Test dataset RW Pearson:", round(rw[0][0],2), ", Spearman:", round(rw[1][0],2))

Test dataset WordSim-353 Pearson: 0.66 , Spearman: 0.7
Test dataset SimVerb-3500 Pearson: 0.24 , Spearman: 0.25
Test dataset MEN Pearson: 0.69 , Spearman: 0.7
Test dataset RW Pearson: 0.48 , Spearman: 0.49


###### 1.3.1.2.2 Word analogy

Word analogy will be tested on two following datasets:

 - **Google analogy test set**, unbalanced: 8,869 semantic and 10,675 syntactic questions, with 20-70 pairs per category; country:capital relation is over 50% of all semantic questions. Relations in the syntactic part largely the same as MSR,
 - **SemEval-2012**, 10,014 questions divided into 10 semantic classes and 79 subclasses prepared for the SemEval-2017 Task 2 (Measuring Degrees of Relational Similarity).

In [0]:
analogy_google = huff_simp_nobigr.wv.evaluate_word_analogies('questions-words.txt')
analogy_SemEval = huff_simp_nobigr.wv.evaluate_word_analogies('semeval.txt')

print("Test dataset Google:", round(analogy_google[0],2))
print("Test dataset SemEval-2012:", round(analogy_SemEval[0],2))

Test dataset Google: 0.44
Test dataset SemEval-2012: 0.01


###### 1.3.1.2.3 Comparison to wordlist

In [0]:
nltk_words = nltk.corpus.words.words('en')
tokens = set(huff_simp_nobigr_vocab.token)
unusual_words = tokens.difference(nltk_words)
print("The number of unusual words:", len(unusual_words))

The number of unusual words: 23437


In [0]:
unusual_words

{'cicilline',
 'angers',
 'edits',
 'talib',
 'pippa',
 'dorothy',
 'disrupts',
 'mc',
 'luxembourg',
 'feeders',
 'accolades',
 'ventured',
 'mins',
 'pendleton',
 'phillippe',
 'sheila',
 'improvising',
 'ideagen',
 'ailey',
 'recanted',
 'outings',
 'juliana',
 'groped',
 'abolitionists',
 'warwick',
 'editing',
 'armstrong',
 'esteban',
 'tasers',
 'widens',
 'somerhalder',
 'emails',
 'boi',
 'elaborating',
 'aussie',
 'resolves',
 'welcomes',
 'juan',
 'jaguars',
 'jen',
 'configured',
 'additives',
 'slabs',
 'aapl',
 'underscoring',
 'utilizes',
 'robocall',
 'jetting',
 'transformed',
 'charlize',
 'athletes',
 'ventures',
 'dempsie',
 'workings',
 'siu',
 'kurtz',
 'staycation',
 'crows',
 'formatted',
 'fuels',
 'adverts',
 'redmond',
 'cardiff',
 'fifa',
 'jamey',
 'edie',
 'focuses',
 'proponents',
 'daycare',
 'critiqued',
 'grids',
 'shakespeare',
 'chlamydia',
 'fios',
 'cato',
 'brussels',
 'ymca',
 'kors',
 'hemings',
 'results',
 'fibroids',
 'conveys',
 'chihuly',
 

In [0]:
f = open('huff_simp_nobigr_UNUSUAL.txt', 'w')
for t in unusual_words:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

#### 1.3.2 Gensim simple pre-processing, with bigrams and trigrams

##### 1.3.2.1 Bigrams generation

In [0]:
stopwords = stopwords.words("english")
print('I will use', len(stopwords), 'stop words from nltk')
common_terms = frozenset(stopwords)
stopwords

I will use 179 stop words from nltk


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [0]:
start_time = time.time()
# threshold for forming the phrases - higher means fewer phrases
# min_count (float, optional) – Ignore all words and bigrams with total collected count lower than this value
bigram_transformer = Phrases(huffpost.tokens,
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 10.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 97.58 seconds


In [0]:
start_time = time.time()
bigram_transformer_second = Phrases(bigram_transformer[huffpost.tokens],
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 30.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 305.43 seconds


Check the bigrams generated by the first round (on one article):

In [0]:
bigrams_stop_low_th = []

for token in bigram_transformer[huffpost.tokens[2384]]:
  if "_" in token:
    bigrams_stop_low_th.append(token)
    
bigrams_stop_low_th

['spencer_platt',
 'via_getty',
 'alexandria_ocasio',
 'joe_crowley',
 'democratic_party',
 'new_york',
 'point_of_view',
 'ben_jealous',
 'bernie_sanders',
 'single_payer',
 'health_care',
 'single_payer',
 'via_getty',
 'role_model',
 'sherrod_brown',
 'democratic_candidates',
 'hard_pressed',
 'hot_button',
 'conor_lamb',
 'th_district',
 'new_york',
 'alexandria_ocasio',
 'joe_crowley',
 'role_model',
 'sherrod_brown',
 'donald_trump',
 'lgbtq_rights',
 'working_class',
 'west_virginia',
 'via_getty',
 'west_virginia',
 'pro_choice',
 'west_virginians',
 'parental_leave',
 'abortion_rights',
 'hillary_clinton',
 'wall_street',
 'kirsten_gillibrand',
 'cory_booker']

Check the bigrams generated by the second round (on one article):

In [0]:
bigrams_stop_low_th_second = []

for token in bigram_transformer_second[huffpost.tokens[2384]]:
  if "_" in token:
    bigrams_stop_low_th_second.append(token)
    
bigrams_stop_low_th_second

['spencer_platt',
 'via_getty',
 'alexandria_ocasio',
 'joe_crowley',
 'democratic_party',
 'new_york',
 'point_of_view',
 'ben_jealous',
 'bernie_sanders',
 'single_payer',
 'health_care',
 'single_payer',
 'ordinary_americans',
 'via_getty',
 'role_model',
 'sherrod_brown',
 'democratic_candidates',
 'hard_pressed',
 'hot_button',
 'conor_lamb',
 'special_election',
 'th_district',
 'new_york',
 'alexandria_ocasio',
 'joe_crowley',
 'role_model',
 'sherrod_brown',
 'donald_trump',
 'lgbtq_rights',
 'socially_conservative',
 'working_class',
 'west_virginia',
 'via_getty',
 'west_virginia',
 'pro_choice',
 'west_virginians',
 'parental_leave',
 'gun_rights',
 'abortion_rights',
 'socially_conservative',
 'hillary_clinton',
 'wall_street',
 'kirsten_gillibrand',
 'cory_booker']

What is new after the second pass?

In [0]:
for t in bigrams_stop_low_th_second:
  if t not in bigrams_stop_low_th:
    print(t)

ordinary_americans
special_election
socially_conservative
gun_rights
socially_conservative


##### 1.3.2.2 Training

In [0]:
start_time = time.time()
huff_simp_bi_trigr = Word2Vec(bigram_transformer_second[huffpost['tokens']],
                      size = 300, window = 8, min_count = 25, workers = 4, sample = 0.00001,
                      sg = 1, compute_loss = True, callbacks = ())
end_time = time.time()

os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_simp_bi_trigr.save("huff_simp_bi_trigr.model")
print("Word2Vec for left news (Huffington Post) for", round((end_time - start_time),2), "seconds")

Word2Vec for left news (Huffington Post) for 1954.4 seconds


##### 1.3.2.3 Analysis of quality

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_simp_bi_trigr = Word2Vec.load("huff_simp_bi_trigr.model")

###### 1.3.2.3.1 Vocabulary

In [0]:
print("Vocabulary length:", len(huff_simp_bi_trigr.wv.vocab), "words")

Vocabulary length: 61006 words


In [0]:
huff_simp_bi_trigr_vocab = {}
for token in huff_simp_bi_trigr.wv.vocab:
  huff_simp_bi_trigr_vocab[token] = huff_simp_bi_trigr.wv.vocab[token].count

huff_simp_bi_trigr_vocab = pd.DataFrame.from_dict(huff_simp_bi_trigr_vocab, orient='index', columns=['frequency'])
huff_simp_bi_trigr_vocab['token'] = huff_simp_bi_trigr_vocab.index
huff_simp_bi_trigr_vocab = huff_simp_bi_trigr_vocab.sort_values(by='frequency', ascending=False)
huff_simp_bi_trigr_vocab = huff_simp_bi_trigr_vocab.reset_index(inplace = False) 
huff_simp_bi_trigr_vocab = huff_simp_bi_trigr_vocab[['token', 'frequency']]

huff_simp_bi_trigr_vocab.head()

,token,frequency
0,the,3303617
1,to,1994385
2,and,1806279
3,of,1592878
4,a,1566619


In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")
huff_simp_bi_trigr_vocab.to_csv(r'huff_simp_bi_trigr_vocab_VOCAB.csv', index = False, header = True)

###### 1.3.2.3.2 Word semantic similarity

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")

In [0]:
wordsim353 = huff_simp_bi_trigr.evaluate_word_pairs('wordsim353.tsv')
simverb3500 = huff_simp_bi_trigr.evaluate_word_pairs('simverb-3500.txt')
men = huff_simp_bi_trigr.evaluate_word_pairs('men.txt')
rw = huff_simp_bi_trigr.evaluate_word_pairs('rw.txt')


print("Test dataset WordSim-353 Pearson:", round(wordsim353[0][0],2), ", Spearman:", round(wordsim353[1][0],2))
print("Test dataset SimVerb-3500 Pearson:", round(simverb3500[0][0],2), ", Spearman:", round(simverb3500[1][0],2))
print("Test dataset MEN Pearson:", round(men[0][0],2), ", Spearman:", round(men[1][0],2))
print("Test dataset RW Pearson:", round(rw[0][0],2), ", Spearman:", round(rw[1][0],2))

Test dataset WordSim-353 Pearson: 0.65 , Spearman: 0.68
Test dataset SimVerb-3500 Pearson: 0.26 , Spearman: 0.26
Test dataset MEN Pearson: 0.7 , Spearman: 0.71
Test dataset RW Pearson: 0.48 , Spearman: 0.49


###### 1.3.2.3.3 Word analogy 

In [0]:
analogy_google = huff_simp_bi_trigr.wv.evaluate_word_analogies('questions-words.txt')
analogy_SemEval = huff_simp_bi_trigr.wv.evaluate_word_analogies('semeval.txt')

print("Test dataset Google:", round(analogy_google[0],2))
print("Test dataset SemEval-2012:", round(analogy_SemEval[0],2))

Test dataset Google: 0.43
Test dataset SemEval-2012: 0.01


In [0]:
accuracy = huff_simp_bi_trigr.wv.accuracy('questions-words.txt', case_insensitive=True)

incorrects = []
for section in range(len(accuracy)):
  for incorrect in accuracy[section]['incorrect']:
    incorrects.append(incorrect)
    
len(incorrects)

10756

In [0]:
incorrect_answers = []

for question in range(len(incorrects)):
  try:
    pos1 = incorrects[question][1].lower()
    pos2 = incorrects[question][2].lower()
    neg = incorrects[question][0].lower()
    answer = huff_simp_bi_trigr.wv.most_similar(positive=[pos1,pos2], negative=[neg], topn=1)[0][0]
    incorrect_answers.append((neg,pos1,pos2,answer))
  except:
    continue
    
incorrect_answers

[('athens', 'greece', 'baghdad', 'iraqi_government'),
 ('athens', 'greece', 'bangkok', 'albania'),
 ('athens', 'greece', 'berlin', 'germany'),
 ('athens', 'greece', 'cairo', 'egypt'),
 ('athens', 'greece', 'havana', 'cuba'),
 ('athens', 'greece', 'helsinki', 'russian_leader'),
 ('athens', 'greece', 'islamabad', 'balochistan'),
 ('athens', 'greece', 'kabul', 'afghan_government'),
 ('athens', 'greece', 'london', 'uk'),
 ('athens', 'greece', 'oslo', 'foreign_ministers'),
 ('athens', 'greece', 'ottawa', 'bulgaria'),
 ('athens', 'greece', 'paris', 'french_president'),
 ('baghdad', 'iraq', 'helsinki', 'russian_leader'),
 ('baghdad', 'iraq', 'islamabad', 'balochistan'),
 ('baghdad', 'iraq', 'london', 'uk'),
 ('baghdad', 'iraq', 'oslo', 'plo'),
 ('baghdad', 'iraq', 'ottawa', 'royal_tour'),
 ('baghdad', 'iraq', 'athens', 'sparta'),
 ('bangkok', 'thailand', 'helsinki', 'russia_interfered'),
 ('bangkok', 'thailand', 'islamabad', 'haqqani_network'),
 ('bangkok', 'thailand', 'kabul', 'afghan'),
 ('

In [0]:
f = open('huff_simp_bi_trigr_ANALOGIES.txt', 'w')
for t in incorrect_answers:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

###### 1.3.2.3.4 Comparison to wordlist

In [0]:
nltk_words = nltk.corpus.words.words('en')
tokens = set(huff_simp_bi_trigr_vocab.token)
unusual_words = tokens.difference(nltk_words)
print("The number of unusual words:", len(unusual_words))

The number of unusual words: 40764


In [0]:
unusual_words

{'v_holder',
 'told_bloomberg',
 'luxury_brands',
 'universal_pictures',
 'cirque_du',
 'angers',
 'edits',
 'alcohol_intake',
 'rose_tico',
 'talib',
 'dorothy',
 'disrupts',
 'mc',
 'luxembourg',
 'feeders',
 'accolades',
 'ventured',
 'alarming_rate',
 'last_tycoon',
 'deutsch_wordpress',
 'mins',
 'became_apparent',
 'tamron_hall',
 'pendleton',
 'cortez_d_n',
 'phillippe',
 'sheila',
 'improvising',
 'pacific_press',
 'south_dakota',
 'crime_stoppers',
 'antibiotic_resistance',
 'ideagen',
 'tax_rate',
 'ailey',
 'tax_bracket',
 'water_bottles',
 'tried_to_convince',
 'smartphone_app',
 'ivanka_and_jared',
 'defunding_planned',
 'statue_of_robert',
 'recanted',
 'outings',
 'confederate_monuments',
 'juliana',
 'hate_crime',
 'groped',
 'abolitionists',
 'dan_scavino',
 'warwick',
 'editing',
 'new_testament',
 'armstrong',
 'esteban',
 'charlottesville_va',
 'cyber_criminals',
 'folic_acid',
 'walter_shaub',
 'tasers',
 'alexis_olympia',
 'widens',
 'professor_geoff',
 'somerhald

In [0]:
f = open('huff_simp_bi_trigr_UNUSUAL.txt', 'w')
for t in unusual_words:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

#### 1.3.3 Manual pre-processing, no bigrams

##### 1.3.3.1 Training

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Data")
huffpost = pd.read_csv("huffington_preprocessed.csv", converters={'text_preprocessed':ast.literal_eval})

In [0]:
start_time = time.time()
huff_manual_nobigr = gensim.models.Word2Vec(huffpost['text_preprocessed'],
                                 size = 300, window = 8, min_count = 25, workers = 4, sample = 0.00001,
                                 sg = 1, compute_loss = True, callbacks = ())
end_time = time.time()

os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_manual_nobigr.save("huff_manual_nobigr.model")
print("Word2Vec for left news (Huffington Post) for", round((end_time - start_time),2), "seconds")

Word2Vec for left news (Huffington Post) for 842.37 seconds


##### 1.3.3.2 Analysis of quality

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_manual_nobigr = Word2Vec.load("huff_manual_nobigr.model")

###### 1.3.2.3.1 Vocabulary

In [0]:
print("Vocabulary length:", len(huff_manual_nobigr.wv.vocab), "words")

Vocabulary length: 48717 words


In [0]:
huff_manual_nobigr_vocab = {}
for token in huff_manual_nobigr.wv.vocab:
  huff_manual_nobigr_vocab[token] = huff_manual_nobigr.wv.vocab[token].count

huff_manual_nobigr_vocab = pd.DataFrame.from_dict(huff_manual_nobigr_vocab, orient='index', columns=['frequency'])
huff_manual_nobigr_vocab['token'] = huff_manual_nobigr_vocab.index
huff_manual_nobigr_vocab = huff_manual_nobigr_vocab.sort_values(by='frequency', ascending=False)
huff_manual_nobigr_vocab = huff_manual_nobigr_vocab.reset_index(inplace = False) 
huff_manual_nobigr_vocab = huff_manual_nobigr_vocab[['token', 'frequency']]

huff_manual_nobigr_vocab.head()

,token,frequency
0,the,3342234
1,to,2007243
2,and,1846525
3,of,1652636
4,a,1566802


In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")
huff_manual_nobigr_vocab.to_csv(r'huff_manual_nobigr_VOCAB.csv', index = False, header = True)

###### 1.3.3.2.2 Word semantic similarity

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")

In [0]:
wordsim353 = huff_manual_nobigr.evaluate_word_pairs('wordsim353.tsv')
simverb3500 = huff_manual_nobigr.evaluate_word_pairs('simverb-3500.txt')
men = huff_manual_nobigr.evaluate_word_pairs('men.txt')
rw = huff_manual_nobigr.evaluate_word_pairs('rw.txt')

print("Test dataset WordSim-353 Pearson:", round(wordsim353[0][0],2), ", Spearman:", round(wordsim353[1][0],2))
print("Test dataset SimVerb-3500 Pearson:", round(simverb3500[0][0],2), ", Spearman:", round(simverb3500[1][0],2))
print("Test dataset MEN Pearson:", round(men[0][0],2), ", Spearman:", round(men[1][0],2))
print("Test dataset RW Pearson:", round(rw[0][0],2), ", Spearman:", round(rw[1][0],2))

Test dataset WordSim-353 Pearson: 0.68 , Spearman: 0.72
Test dataset SimVerb-3500 Pearson: 0.24 , Spearman: 0.24
Test dataset MEN Pearson: 0.69 , Spearman: 0.7
Test dataset RW Pearson: 0.47 , Spearman: 0.48


###### 1.3.3.2.3 Word analogy 

In [0]:
analogy_google = huff_manual_nobigr.wv.evaluate_word_analogies('questions-words.txt')
analogy_SemEval = huff_manual_nobigr.wv.evaluate_word_analogies('semeval.txt')

print("Test dataset Google:", round(analogy_google[0],2))
print("Test dataset SemEval-2012:", round(analogy_SemEval[0],2))

Test dataset Google: 0.42
Test dataset SemEval-2012: 0.01


###### 1.3.3.2.4 Comparison to wordlist

In [0]:
nltk_words = nltk.corpus.words.words('en')
tokens = set(huff_manual_nobigr_vocab.token)
unusual_words = tokens.difference(nltk_words)
print("The number of unusual words:", len(unusual_words))

The number of unusual words: 28555


In [0]:
unusual_words

{'cicilline',
 'angers',
 'edits',
 "60's",
 'talib',
 "nigeria's",
 'pippa',
 'dorothy',
 'disrupts',
 'mc',
 'luxembourg',
 "ossoff's",
 'agreed-upon',
 'feeders',
 'accolades',
 'ventured',
 'mins',
 'pendleton',
 "year's",
 'phillippe',
 "nature's",
 'fifty-two thousand',
 'sheila',
 'well-paid',
 'improvising',
 'coal-fired',
 'ideagen',
 "committee's",
 'ailey',
 'recanted',
 'outings',
 'juliana',
 'groped',
 'abolitionists',
 'warwick',
 'editing',
 'hivaids',
 'armstrong',
 'esteban',
 'tasers',
 'widens',
 'somerhalder',
 'emails',
 'boi',
 'short-lived',
 'elaborating',
 'aussie',
 'resolves',
 'welcomes',
 'juan',
 'jaguars',
 'jen',
 'configured',
 'additives',
 '9-year-old',
 'slabs',
 'aapl',
 'underscoring',
 'utilizes',
 'robocall',
 'jetting',
 'transformed',
 'charlize',
 'athletes',
 'ventures',
 "anybody's",
 'dempsie',
 'workings',
 'siu',
 'kurtz',
 'staycation',
 'crows',
 'formatted',
 'fuels',
 '2015′s',
 'one hundred and thirty-one',
 'mid-atlantic',
 "manage

In [0]:
f = open('huff_manual_nobigr_UNUSUAL.txt', 'w')
for t in unusual_words:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

#### 1.3.4 Manual pre-processing, with bigrams and trigrams

##### 1.3.4.1 Bigrams generation

In [0]:
stopwords = stopwords.words("english")
common_terms = frozenset(stopwords)

In [0]:
start_time = time.time()
# threshold for forming the phrases - higher means fewer phrases
# min_count (float, optional) – Ignore all words and bigrams with total collected count lower than this value
bigram_transformer = Phrases(huffpost.text_preprocessed,
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 10.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 101.05 seconds


In [0]:
start_time = time.time()
bigram_transformer_second = Phrases(bigram_transformer[huffpost.text_preprocessed],
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 30.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 305.49 seconds


Check the bigrams generated by the first round (on one article):

In [0]:
bigrams_stop_low_th = []

for token in bigram_transformer[huffpost.text_preprocessed[1]]:
  if "_" in token:
    bigrams_stop_low_th.append(token)
    
bigrams_stop_low_th

['spencer_platt',
 'via_getty',
 'alexandria_ocasio-cortez',
 'joe_crowley',
 'social_issues',
 'democratic_party',
 'new_york',
 'republican_incumbents',
 'party_leaders',
 'point_of_view',
 'local_officials',
 'ben_jealous',
 'bernie_sanders',
 'single-payer_health',
 'nominee_for_governor',
 'dear_god',
 'forty_years',
 'ordinary_americans',
 'two_years',
 'losing_their_jobs',
 'via_getty',
 'role_model',
 'sherrod_brown',
 'democratic_candidates',
 'social_issues',
 'conor_lamb',
 'special_election',
 "pennsylvania's_18th",
 'nearly_twenty',
 'even_bother',
 "new_york's",
 'alexandria_ocasio-cortez',
 'joe_crowley',
 'social_issues',
 'role_model',
 'sherrod_brown',
 're-election_campaign',
 'donald_trump',
 'social_issues',
 'lgbtq_rights',
 'social_issues',
 'socially_conservative',
 'working-class_voters',
 'west_virginia',
 'house_seat',
 'state_senator',
 'voted_for_trump',
 'via_getty',
 'west_virginia',
 'five_two thousand and eighteen',
 'social_issues',
 'told_the_crowd',


Check the bigrams generated by the second round (on one article):

In [0]:
bigrams_stop_low_th_second = []

for token in bigram_transformer_second[huffpost.text_preprocessed[1]]:
  if "_" in token:
    bigrams_stop_low_th_second.append(token)
    
bigrams_stop_low_th_second

['spencer_platt',
 'via_getty',
 'alexandria_ocasio-cortez',
 'joe_crowley',
 'economic_and_social',
 'democratic_party',
 'new_york',
 'republican_incumbents',
 'point_of_view',
 'local_officials',
 'ben_jealous',
 'bernie_sanders',
 'single-payer_health',
 'dear_god',
 'forty_years',
 'ordinary_americans',
 'two_years',
 'via_getty',
 'role_model',
 'sherrod_brown',
 'democratic_candidates',
 'social_issues',
 'conor_lamb',
 'special_election',
 "pennsylvania's_18th",
 'nearly_twenty',
 "new_york's",
 'alexandria_ocasio-cortez',
 'joe_crowley',
 'economic_and_social',
 'role_model',
 'sherrod_brown',
 're-election_campaign',
 'donald_trump',
 'social_issues',
 'lgbtq_rights',
 'social_issues',
 'socially_conservative',
 'west_virginia',
 'house_seat',
 'state_senator',
 'via_getty',
 'west_virginia',
 'social_issues',
 'told_the_crowd',
 'west_virginians',
 'social_issues',
 'parental_leave',
 'social_issues',
 'gun_rights',
 'abortion_rights',
 'socially_conservative',
 'parts_of_th

What is new after the second pass?

In [0]:
for t in bigrams_stop_low_th_second:
  if t not in bigrams_stop_low_th:
    print(t)

economic_and_social
economic_and_social


Which bigrams generated from tokens created by manual pre-processing were not captured by tokens created by gensim pre-processing?

In [0]:
for t in bigrams_stop_low_th_second:
  if t not in bigrams_gensim:
    print(t)

alexandria_ocasio-cortez
economic_and_social
republican_incumbents
local_officials
single-payer_health
dear_god
forty_years
two_years
social_issues
pennsylvania's_18th
nearly_twenty
new_york's
alexandria_ocasio-cortez
economic_and_social
re-election_campaign
social_issues
social_issues
house_seat
state_senator
social_issues
told_the_crowd
social_issues
social_issues
parts_of_the_country


Which bigrams generated from tokens created by gensim pre-processing were not captured by tokens created by  manual pre-processing?

In [0]:
for t in bigrams_gensim:
  if t not in bigrams_stop_low_th_second:
    print(t)

alexandria_ocasio
single_payer
health_care
single_payer
hard_pressed
hot_button
th_district
alexandria_ocasio
working_class
pro_choice
wall_street


##### 1.3.4.2 Training

In [0]:
start_time = time.time()
huff_manual_bi_trigr = gensim.models.Word2Vec(bigram_transformer_second[huffpost['text_preprocessed']],
                                 size = 300, window = 8, min_count = 25, workers = 4, sample = 0.00001,
                                 sg = 1, compute_loss = True, callbacks = ())
end_time = time.time()

os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_manual_bi_trigr.save("huff_manual_bi_trigr.model")
print("Word2Vec for left news (Huffington Post) for", round((end_time - start_time),2), "seconds")

##### 1.3.4.3 Analysis of quality

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_manual_bi_trigr = Word2Vec.load("huff_manual_bi_trigr.model")

In [0]:
huff_manual_bi_trigr.wv.most_similar("two thousand and eight", topn=50)

[('two thousand and thirteen', 0.6706548929214478),
 ('two thousand and seven', 0.6697640419006348),
 ('two thousand and twelve', 0.6572495698928833),
 ('two thousand and ten', 0.6554303169250488),
 ('two thousand and nine', 0.6463243961334229),
 ('two thousand and fourteen', 0.6388760209083557),
 ('two thousand and eleven', 0.6286214590072632),
 ('two thousand and four', 0.6080456376075745),
 ('two thousand and two', 0.6079401969909668),
 ('two thousand and six', 0.581938624382019),
 ('two thousand and fifteen', 0.5682384967803955),
 ('two thousand and three', 0.5625367760658264),
 ('one thousand, nine hundred and ninety-eight', 0.5477012991905212),
 ('one thousand, nine hundred and ninety-six', 0.5471971035003662),
 ('two thousand and sixteen', 0.541264533996582),
 ('two thousand and one', 0.5385209321975708),
 ('two thousand and five', 0.5371018648147583),
 ('mid-2000s', 0.5242598056793213),
 ('one thousand, nine hundred and ninety-two', 0.5196840763092041),
 ('one thousand, nine hu

In [0]:
huff_manual_bi_trigr.wv.most_similar("two thousand and one", topn=20)

[('two thousand and three', 0.59914231300354),
 ('sept_eleven', 0.5836012363433838),
 ('two thousand and two', 0.5692138671875),
 ('two thousand and eight', 0.5385209321975708),
 ('one thousand, nine hundred and ninety-eight', 0.513603925704956),
 ('two thousand and seven', 0.502682089805603),
 ('one thousand, nine hundred and ninety-one', 0.496537446975708),
 ('two thousand and four', 0.4934804439544678),
 ('one thousand, nine hundred and ninety-three', 0.49017995595932007),
 ('two thousand and ten', 0.48799315094947815),
 ('two thousand and eleven', 0.4876994490623474),
 ('nine hundred and eleven_terrorist', 0.4818481206893921),
 ('transited', 0.47786012291908264),
 ('two thousand and nine', 0.47667065262794495),
 ('then-president_george', 0.46489107608795166),
 ('one thousand, nine hundred and seventy-five', 0.4640851616859436),
 ('two thousand and one_terrorist', 0.463304340839386),
 ('nine hundred and eleven_attacks', 0.45798760652542114),
 ('one thousand, nine hundred and eighty-

###### 1.3.2.3.1 Vocabulary

In [0]:
print("Vocabulary length:", len(huff_manual_bi_trigr.wv.vocab), "words")

Vocabulary length: 69357 words


In [0]:
huff_manual_bi_trigr_vocab = {}
for token in huff_manual_bi_trigr.wv.vocab:
  huff_manual_bi_trigr_vocab[token] = huff_manual_bi_trigr.wv.vocab[token].count

huff_manual_bi_trigr_vocab = pd.DataFrame.from_dict(huff_manual_bi_trigr_vocab, orient='index', columns=['frequency'])
huff_manual_bi_trigr_vocab['token'] = huff_manual_bi_trigr_vocab.index
huff_manual_bi_trigr_vocab = huff_manual_bi_trigr_vocab.sort_values(by='frequency', ascending=False)
huff_manual_bi_trigr_vocab = huff_manual_bi_trigr_vocab.reset_index(inplace = False) 
huff_manual_bi_trigr_vocab = huff_manual_bi_trigr_vocab[['token', 'frequency']]

huff_manual_bi_trigr_vocab.head()

,token,frequency
0,the,3283484
1,to,1983094
2,and,1786942
3,of,1576175
4,a,1546905


In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")
huff_manual_bi_trigr_vocab.to_csv(r'huff_manual_bi_trigr_VOCAB.csv', index = False, header = True)

###### 1.3.4.3.2 Word semantic similarity

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")

In [0]:
wordsim353 = huff_manual_bi_trigr.evaluate_word_pairs('wordsim353.tsv')
simverb3500 = huff_manual_bi_trigr.evaluate_word_pairs('simverb-3500.txt')
men = huff_manual_bi_trigr.evaluate_word_pairs('men.txt')
rw = huff_manual_bi_trigr.evaluate_word_pairs('rw.txt')

print("Test dataset WordSim-353 Pearson:", round(wordsim353[0][0],2), ", Spearman:", round(wordsim353[1][0],2))
print("Test dataset SimVerb-3500 Pearson:", round(simverb3500[0][0],2), ", Spearman:", round(simverb3500[1][0],2))
print("Test dataset MEN Pearson:", round(men[0][0],2), ", Spearman:", round(men[1][0],2))
print("Test dataset RW Pearson:", round(rw[0][0],2), ", Spearman:", round(rw[1][0],2))

Test dataset WordSim-353 Pearson: 0.66 , Spearman: 0.71
Test dataset SimVerb-3500 Pearson: 0.25 , Spearman: 0.25
Test dataset MEN Pearson: 0.7 , Spearman: 0.71
Test dataset RW Pearson: 0.48 , Spearman: 0.49


###### 1.3.4.3.3 Word analogy 

In [0]:
analogy_google = huff_manual_bi_trigr.wv.evaluate_word_analogies('questions-words.txt')
analogy_SemEval = huff_manual_bi_trigr.wv.evaluate_word_analogies('semeval.txt')

print("Test dataset Google:", round(analogy_google[0],2))
print("Test dataset SemEval-2012:", round(analogy_SemEval[0],2))

Test dataset Google: 0.41
Test dataset SemEval-2012: 0.01


In [0]:
accuracy = huff_manual_bi_trigr.wv.accuracy('questions-words.txt', case_insensitive=True)

incorrects = []
for section in range(len(accuracy)):
  for incorrect in accuracy[section]['incorrect']:
    incorrects.append(incorrect)
    
len(incorrects)

10052

In [0]:
incorrect_answers = []

for question in range(len(incorrects)):
  try:
    pos1 = incorrects[question][1].lower()
    pos2 = incorrects[question][2].lower()
    neg = incorrects[question][0].lower()
    answer = huff_manual_bi_trigr.wv.most_similar(positive=[pos1,pos2], negative=[neg], topn=1)[0][0]
    incorrect_answers.append((neg,pos1,pos2,answer))
  except:
    continue
    
incorrect_answers

[('athens', 'greece', 'bangkok', 'albania'),
 ('athens', 'greece', 'helsinki', 'russian_leader'),
 ('athens', 'greece', 'kabul', 'afghan_government'),
 ('athens', 'greece', 'london', 'uk'),
 ('athens', 'greece', 'oslo', 'foreign_ministers'),
 ('athens', 'greece', 'paris', 'french_president'),
 ('baghdad', 'iraq', 'helsinki', 'russian_leader'),
 ('baghdad', 'iraq', 'london', 'uk'),
 ('baghdad', 'iraq', 'oslo', 'plo'),
 ('bangkok', 'thailand', 'berlin', 'german'),
 ('bangkok', 'thailand', 'helsinki', 'russia_interfered'),
 ('bangkok', 'thailand', 'kabul', 'afghan'),
 ('bangkok', 'thailand', 'london', 'uk'),
 ('bangkok', 'thailand', 'oslo', 'arafat'),
 ('beijing', 'china', 'helsinki', 'russian_leader'),
 ('beijing', 'china', 'kabul', 'afghan'),
 ('beijing', 'china', 'london', 'uk'),
 ('beijing', 'china', 'oslo', 'oslo_norway'),
 ('berlin', 'germany', 'helsinki', 'russian_leader'),
 ('berlin', 'germany', 'kabul', 'afghans'),
 ('berlin', 'germany', 'london', 'united_kingdom'),
 ('berlin', '

In [0]:
f = open('huff_manual_bi_trigr_ANALOGIES.txt', 'w')
for t in incorrect_answers:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

###### 1.3.4.3.4 Comparison to wordlist

In [0]:
nltk_words = nltk.corpus.words.words('en')
tokens = set(huff_manual_bi_trigr_vocab.token)
unusual_words = tokens.difference(nltk_words)
print("The number of unusual words:", len(unusual_words))

The number of unusual words: 49401


In [0]:
unusual_words

{'v_holder',
 'told_bloomberg',
 'cicilline',
 'cirque_du',
 'universal_pictures',
 'alcohol_intake',
 'rose_tico',
 'dorothy',
 'disrupts',
 'mc',
 "ossoff's",
 'agreed-upon',
 'feeders',
 'ventured',
 'alarming_rate',
 'last_tycoon',
 'mins',
 "year's",
 'improvising',
 'antibiotic_resistance',
 'coal-fired',
 "committee's",
 'tried_to_convince',
 'tax_bracket',
 'water_bottles',
 'smartphone_app',
 'statue_of_robert',
 'throwing_away',
 'outings',
 'confederate_monuments',
 'juliana',
 'groped',
 'dan_scavino',
 'warwick',
 'editing',
 'hivaids',
 'armstrong',
 'folic_acid',
 'alexis_olympia',
 'boi',
 'short-lived',
 'elaborating',
 'resolves',
 'welcomes',
 'jaguars',
 'jen',
 'additives',
 'slabs',
 'aapl',
 'robocall',
 'transgender_woman',
 'transformed',
 'acting_administrator',
 'environmental_regulations',
 'every_single',
 'immediately_return',
 'basic_tenets',
 'absolutely_unacceptable',
 'workings',
 'nobody_cares',
 'fuels',
 'lgbtq_americans',
 'illegal_activity',
 'nea

In [0]:
f = open('huff_manual_bi_trigr_UNUSUAL.txt', 'w')
for t in unusual_words:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

### 1.4 Final pre-processing

#### 1.4.1 Include title in the text

In [0]:
huffpost['text_preprocessed'] = huffpost["title"].astype(str) + ". " + huffpost["text"]

#### 1.4.2 Remove links and nicknames

In [0]:
http_number, www_number, pic_number, at_number = 0, 0, 0, 0
for row in huffpost.text_preprocessed:
  if "http" in row:
    http_number += 1
  if 'www' in row:
    www_number += 1
  if 'pic.' in row:
    pic_number += 1
  if '@' in row:
    at_number += 1

print('The number of articles containing "http" in the text', http_number)
print('The number of articles containing "www" in the text', www_number)
print('The number of articles containing "pic." in the text', pic_number)
print('The number of articles containing "@" in the text', at_number)

The number of articles containing "http" in the text 7075
The number of articles containing "www" in the text 2870
The number of articles containing "pic." in the text 9497
The number of articles containing "@" in the text 18196


In [0]:
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'http\S+', '', row.text_preprocessed), axis=1) # including https
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'www\.\S+', '', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'pic\.\S+', '', row.text_preprocessed), axis=1)
huffpost['text_preprocessed'] = huffpost.apply(lambda row: re.sub(r'@\S+', '', row.text_preprocessed), axis=1)

#### 1.4.3 Gensim simple pre-processing

In [0]:
start_time = time.time()
huffpost['tokens'] = huffpost.apply(lambda row: gensim.utils.simple_preprocess(row.text_preprocessed, 
                                                                               deacc=False, min_len=1, max_len=28),
                                    axis=1)
end_time = time.time()
print("huffpost gensim simple_preprocess", round((end_time - start_time),2), "seconds")

huffpost gensim simple_preprocess 66.7 seconds


In [0]:
huffpost['words'] = huffpost.apply(lambda row: len(row.tokens), axis=1)

print("Total number of words for training:", sum(huffpost['words']))
print("Average number of words per article:", round(statistics.mean(huffpost['words']),0))

Total number of words for training: 68172239
Average number of words per article: 674.0


### 1.5 Final word embeddings

#### 1.5.1 Bigrams generation

##### 1.5.1.1 Thresholds 10 and 30

Check the scores of a sample of real bigrams

In [0]:
print("Very frequent:")
print("new_york", original_scorer(bigram_transformer.vocab[b'new'], bigram_transformer.vocab[b'york'], 
                    bigram_transformer.vocab[b'new_york'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("climate_change", original_scorer(bigram_transformer.vocab[b'climate'], bigram_transformer.vocab[b'change'], 
                    bigram_transformer.vocab[b'climate_change'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("left_wing", original_scorer(bigram_transformer.vocab[b'left'], bigram_transformer.vocab[b'wing'], 
                    bigram_transformer.vocab[b'left_wing'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("civil_rights", original_scorer(bigram_transformer.vocab[b'civil'], bigram_transformer.vocab[b'rights'], 
                    bigram_transformer.vocab[b'civil_rights'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("human_rights", original_scorer(bigram_transformer.vocab[b'human'], bigram_transformer.vocab[b'rights'], 
                    bigram_transformer.vocab[b'human_rights'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("sexual_harassment", original_scorer(bigram_transformer.vocab[b'sexual'], bigram_transformer.vocab[b'harassment'], 
                    bigram_transformer.vocab[b'sexual_harassment'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("anti_semitic", original_scorer(bigram_transformer.vocab[b'anti'], bigram_transformer.vocab[b'semitic'], 
                    bigram_transformer.vocab[b'anti_semitic'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("anti_lgbtq", original_scorer(bigram_transformer.vocab[b'anti'], bigram_transformer.vocab[b'lgbtq'], 
                    bigram_transformer.vocab[b'anti_lgbtq'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("Medium frequent:")
print("self_promotion", original_scorer(bigram_transformer.vocab[b'self'], bigram_transformer.vocab[b'promotion'], 
                    bigram_transformer.vocab[b'self_promotion'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("tax_fraud", original_scorer(bigram_transformer.vocab[b'tax'], bigram_transformer.vocab[b'fraud'], 
                    bigram_transformer.vocab[b'tax_fraud'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("bike_share", original_scorer(bigram_transformer.vocab[b'bike'], bigram_transformer.vocab[b'share'], 
                    bigram_transformer.vocab[b'bike_share'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("illegal_aliens", original_scorer(bigram_transformer.vocab[b'illegal'], bigram_transformer.vocab[b'aliens'], 
                    bigram_transformer.vocab[b'illegal_aliens'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("pro_choice", original_scorer(bigram_transformer.vocab[b'pro'], bigram_transformer.vocab[b'choice'], 
                    bigram_transformer.vocab[b'pro_choice'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("Rare:")
print("limited_liability", original_scorer(bigram_transformer.vocab[b'limited'], bigram_transformer.vocab[b'liability'], 
                    bigram_transformer.vocab[b'limited_liability'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("audio_recordings", original_scorer(bigram_transformer.vocab[b'audio'], bigram_transformer.vocab[b'recordings'], 
                    bigram_transformer.vocab[b'audio_recordings'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("auto_pilot", original_scorer(bigram_transformer.vocab[b'auto'], bigram_transformer.vocab[b'pilot'], 
                    bigram_transformer.vocab[b'auto_pilot'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("westminster_abbey", original_scorer(bigram_transformer.vocab[b'westminster'], bigram_transformer.vocab[b'abbey'], 
                    bigram_transformer.vocab[b'westminster_abbey'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

Very frequent:
new_york 154.48516218496692
climate_change 250.0298531202226
left_wing 101.62679355675907
civil_rights 374.46785059152614
human_rights 163.22080595400888
sexual_harassment 532.5095309904502
anti_semitic 1353.1335189491524
anti_lgbtq 102.27561044843367
Medium frequent:
self_promotion 36.65566785642789
tax_fraud 17.75379120420146
bike_share 41.50921181045425
illegal_aliens 662.5123260623196
pro_choice 116.91339719655505
Rare:
limited_liability 53.260706472796244
audio_recordings 268.56713763834347
auto_pilot 91.39903352407279
westminster_abbey 6614.475009278733


Check the scores of a sample of wrong bigrams

In [0]:
print("Very frequent:")
print("told_huffpost", original_scorer(bigram_transformer.vocab[b'told'], bigram_transformer.vocab[b'huffpost'], 
                    bigram_transformer.vocab[b'told_huffpost'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("last_week", original_scorer(bigram_transformer.vocab[b'last'], bigram_transformer.vocab[b'week'], 
                    bigram_transformer.vocab[b'last_week'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("make_sure", original_scorer(bigram_transformer.vocab[b'make'], bigram_transformer.vocab[b'sure'], 
                    bigram_transformer.vocab[b'make_sure'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("reminds_us", original_scorer(bigram_transformer.vocab[b'reminds'], bigram_transformer.vocab[b'us'], 
                    bigram_transformer.vocab[b'reminds_us'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("every_day", original_scorer(bigram_transformer.vocab[b'every'], bigram_transformer.vocab[b'day'], 
                    bigram_transformer.vocab[b'every_day'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("Medium frequent:")
print("moving_parts", original_scorer(bigram_transformer.vocab[b'moving'], bigram_transformer.vocab[b'parts'], 
                    bigram_transformer.vocab[b'moving_parts'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("former_employer", original_scorer(bigram_transformer.vocab[b'former'], bigram_transformer.vocab[b'employer'], 
                    bigram_transformer.vocab[b'former_employer'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("rising_temperatures", original_scorer(bigram_transformer.vocab[b'rising'], bigram_transformer.vocab[b'temperatures'], 
                    bigram_transformer.vocab[b'rising_temperatures'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("documents_related", original_scorer(bigram_transformer.vocab[b'documents'], bigram_transformer.vocab[b'related'], 
                    bigram_transformer.vocab[b'documents_related'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("Rare:")
print("incorrect_information", original_scorer(bigram_transformer.vocab[b'incorrect'], bigram_transformer.vocab[b'information'], 
                    bigram_transformer.vocab[b'incorrect_information'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("christmas_presents", original_scorer(bigram_transformer.vocab[b'christmas'], bigram_transformer.vocab[b'presents'], 
                    bigram_transformer.vocab[b'christmas_presents'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("primary_objective", original_scorer(bigram_transformer.vocab[b'primary'], bigram_transformer.vocab[b'objective'], 
                    bigram_transformer.vocab[b'primary_objective'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

print("rental_income", original_scorer(bigram_transformer.vocab[b'rental'], bigram_transformer.vocab[b'income'], 
                    bigram_transformer.vocab[b'rental_income'], len(bigram_transformer.vocab), 
                    bigram_transformer.min_count, bigram_transformer.corpus_word_count))

Very frequent:
told_huffpost 146.77794285285
last_week 93.78138720959296
make_sure 79.4013327979774
reminds_us 75.16414435876845
every_day 40.89132798249094
Medium frequent:
moving_parts 23.061485884263444
former_employer 21.37805797530887
rising_temperatures 186.08611863155303
documents_related 34.38382672744311
Rare:
incorrect_information 20.219643476422558
christmas_presents 25.367923561439092
primary_objective 25.254077557914783
rental_income 19.52055580758316


##### 1.5.1.2 Thresholds 90 and 120

In [0]:
stopwords = stopwords.words("english")
print('I will use', len(stopwords), 'stop words from nltk')
common_terms = frozenset(stopwords)

I will use 179 stop words from nltk


In [0]:
start_time = time.time()
# threshold for forming the phrases - higher means fewer phrases
# min_count (float, optional) – Ignore all words and bigrams with total collected count lower than this value
bigram_transformer = Phrases(huffpost.tokens,
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 90.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 81.32 seconds


In [0]:
start_time = time.time()
bigram_transformer_second = Phrases(bigram_transformer[huffpost.tokens],
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 120.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 261.58 seconds


Check the bigrams generated by the first round (on one article):

In [0]:
bigrams_stop_low_th = []

for token in bigram_transformer[huffpost.tokens[2384]]:
  if "_" in token:
    bigrams_stop_low_th.append(token)
    
bigrams_stop_low_th

['spencer_platt',
 'via_getty',
 'alexandria_ocasio',
 'joe_crowley',
 'democratic_party',
 'new_york',
 'ben_jealous',
 'bernie_sanders',
 'single_payer',
 'health_care',
 'single_payer',
 'via_getty',
 'sherrod_brown',
 'hot_button',
 'conor_lamb',
 'new_york',
 'alexandria_ocasio',
 'joe_crowley',
 'sherrod_brown',
 'donald_trump',
 'west_virginia',
 'via_getty',
 'west_virginia',
 'pro_choice',
 'west_virginians',
 'parental_leave',
 'hillary_clinton',
 'wall_street',
 'kirsten_gillibrand',
 'cory_booker']

Check the bigrams generated by the second round (on one article):

In [0]:
bigrams_stop_low_th_second = []

for token in bigram_transformer_second[huffpost.tokens[2384]]:
  if "_" in token:
    bigrams_stop_low_th_second.append(token)
    
bigrams_stop_low_th_second

['spencer_platt',
 'via_getty',
 'alexandria_ocasio',
 'joe_crowley',
 'democratic_party',
 'new_york',
 'ben_jealous',
 'bernie_sanders',
 'single_payer',
 'health_care',
 'single_payer',
 'via_getty',
 'sherrod_brown',
 'hot_button',
 'conor_lamb',
 'th_district',
 'new_york',
 'alexandria_ocasio',
 'joe_crowley',
 'sherrod_brown',
 'donald_trump',
 'lgbtq_rights',
 'west_virginia',
 'via_getty',
 'west_virginia',
 'west_virginians',
 'parental_leave',
 'abortion_rights',
 'hillary_clinton',
 'wall_street',
 'kirsten_gillibrand',
 'cory_booker']

What is new after the second pass?

In [0]:
for t in bigrams_stop_low_th_second:
  if t not in bigrams_stop_low_th:
    print(t)

th_district
lgbtq_rights
abortion_rights


#### 1.5.2 Training

In [0]:
start_time = time.time()
huff_simp_bi_trigr10 = Word2Vec(bigram_transformer_second[huffpost['tokens']],
                      size = 300, window = 8, min_count = 25, workers = 4, sample = 0.00001,
                      sg = 1, compute_loss = True, callbacks = (), iter = 10)
end_time = time.time()

os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_simp_bi_trigr10.save("huff_simp_bi_trigr10.model")
print("Word2Vec for left news (Huffington Post) for", round((end_time - start_time),2), "seconds")

Word2Vec for left news (Huffington Post) for 3128.95 seconds


#### 1.5.3 Analysis of quality

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Models")
huff_simp_bi_trigr10 = Word2Vec.load("huff_simp_bi_trigr10.model")

##### 1.5.3.1 Vocabulary

In [0]:
print("Vocabulary length:", len(huff_simp_bi_trigr10.wv.vocab), "words")

Vocabulary length: 53223 words


In [0]:
huff_simp_bi_trigr10_vocab = {}
for token in huff_simp_bi_trigr10.wv.vocab:
  huff_simp_bi_trigr10_vocab[token] = huff_simp_bi_trigr10.wv.vocab[token].count

huff_simp_bi_trigr10_vocab = pd.DataFrame.from_dict(huff_simp_bi_trigr10_vocab, orient='index', columns=['frequency'])
huff_simp_bi_trigr10_vocab['token'] = huff_simp_bi_trigr10_vocab.index
huff_simp_bi_trigr10_vocab = huff_simp_bi_trigr10_vocab.sort_values(by='frequency', ascending=False)
huff_simp_bi_trigr10_vocab = huff_simp_bi_trigr10_vocab.reset_index(inplace = False) 
huff_simp_bi_trigr10_vocab = huff_simp_bi_trigr10_vocab[['token', 'frequency']]

huff_simp_bi_trigr10_vocab.head()

,token,frequency
0,the,3331072
1,to,2007089
2,and,1824231
3,of,1622205
4,a,1573876


In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")
huff_simp_bi_trigr10_vocab.to_csv(r'huff_simp_bi_trigr10_VOCAB.csv', index = False, header = True)

##### 1.5.3.2 Word semantic similarity

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")

In [0]:
wordsim353 = huff_simp_bi_trigr10.evaluate_word_pairs('wordsim353.tsv')
simverb3500 = huff_simp_bi_trigr10.evaluate_word_pairs('simverb-3500.txt')
men = huff_simp_bi_trigr10.evaluate_word_pairs('men.txt')
rw = huff_simp_bi_trigr10.evaluate_word_pairs('rw.txt')


print("Test dataset WordSim-353 Pearson:", round(wordsim353[0][0],2), ", Spearman:", round(wordsim353[1][0],2))
print("Test dataset SimVerb-3500 Pearson:", round(simverb3500[0][0],2), ", Spearman:", round(simverb3500[1][0],2))
print("Test dataset MEN Pearson:", round(men[0][0],2), ", Spearman:", round(men[1][0],2))
print("Test dataset RW Pearson:", round(rw[0][0],2), ", Spearman:", round(rw[1][0],2))

Test dataset WordSim-353 Pearson: 0.65 , Spearman: 0.68
Test dataset SimVerb-3500 Pearson: 0.22 , Spearman: 0.24
Test dataset MEN Pearson: 0.71 , Spearman: 0.72
Test dataset RW Pearson: 0.47 , Spearman: 0.49


##### 1.5.3.3 Word analogy 

In [0]:
analogy_google = huff_simp_bi_trigr10.wv.evaluate_word_analogies('questions-words.txt')
#analogy_SemEval = huff_simp_bi_trigr10.wv.evaluate_word_analogies('semeval.txt')

print("Test dataset Google:", round(analogy_google[0],2))
#print("Test dataset SemEval-2012:", round(analogy_SemEval[0],2))

Test dataset Google: 0.5


In [0]:
accuracy = huff_simp_bi_trigr10.wv.accuracy('questions-words.txt', case_insensitive=True)

incorrects = []
for section in range(len(accuracy)):
  for incorrect in accuracy[section]['incorrect']:
    incorrects.append(incorrect)
    
len(incorrects)

10624

In [0]:
incorrect_answers = []

for question in range(len(incorrects)):
  try:
    pos1 = incorrects[question][1].lower()
    pos2 = incorrects[question][2].lower()
    neg = incorrects[question][0].lower()
    answer = huff_simp_bi_trigr10.wv.most_similar(positive=[pos1,pos2], negative=[neg], topn=1)[0][0]
    incorrect_answers.append((neg,pos1,pos2,answer))
  except:
    continue
    
incorrect_answers

[('athens', 'greece', 'helsinki', 'putin_in_helsinki'),
 ('athens', 'greece', 'kabul', 'afghan'),
 ('athens', 'greece', 'london', 'uk'),
 ('athens', 'greece', 'oslo', 'eu'),
 ('baghdad', 'iraq', 'helsinki', 'putin_in_helsinki'),
 ('baghdad', 'iraq', 'london', 'uk'),
 ('bangkok', 'thailand', 'helsinki', 'putin_in_helsinki'),
 ('bangkok', 'thailand', 'kabul', 'afghan'),
 ('bangkok', 'thailand', 'london', 'british'),
 ('bangkok', 'thailand', 'oslo', 'arafat'),
 ('bangkok', 'thailand', 'paris', 'french'),
 ('bangkok', 'thailand', 'baghdad', 'iraqi'),
 ('beijing', 'china', 'helsinki', 'putin_in_helsinki'),
 ('beijing', 'china', 'london', 'uk'),
 ('beijing', 'china', 'baghdad', 'erbil'),
 ('berlin', 'germany', 'helsinki', 'putin_in_helsinki'),
 ('berlin', 'germany', 'kabul', 'afghan'),
 ('berlin', 'germany', 'london', 'britain'),
 ('cairo', 'egypt', 'helsinki', 'putin_in_helsinki'),
 ('cairo', 'egypt', 'kabul', 'afghan'),
 ('cairo', 'egypt', 'london', 'british'),
 ('cairo', 'egypt', 'oslo', 

[('athens', 'greece', 'helsinki', 'putin_in_helsinki'),
 ('athens', 'greece', 'kabul', 'afghan'),
 ('athens', 'greece', 'london', 'uk'),
 ('athens', 'greece', 'oslo', 'eu'),
 ('baghdad', 'iraq', 'helsinki', 'putin_in_helsinki'),
 ('baghdad', 'iraq', 'london', 'uk'),
 ('bangkok', 'thailand', 'helsinki', 'putin_in_helsinki'),
 ('bangkok', 'thailand', 'kabul', 'afghan'),
 ('bangkok', 'thailand', 'london', 'british'),
 ('bangkok', 'thailand', 'oslo', 'arafat'),
 ('bangkok', 'thailand', 'paris', 'french'),
 ('bangkok', 'thailand', 'baghdad', 'iraqi'),
 ('beijing', 'china', 'helsinki', 'putin_in_helsinki'),
 ('beijing', 'china', 'london', 'uk'),
 ('beijing', 'china', 'baghdad', 'erbil'),
 ('berlin', 'germany', 'helsinki', 'putin_in_helsinki'),
 ('berlin', 'germany', 'kabul', 'afghan'),
 ('berlin', 'germany', 'london', 'britain'),
 ('cairo', 'egypt', 'helsinki', 'putin_in_helsinki'),
 ('cairo', 'egypt', 'kabul', 'afghan'),
 ('cairo', 'egypt', 'london', 'british'),
 ('cairo', 'egypt', 'oslo', 

In [0]:
f = open('huff_simp_bi_trigr10_ANALOGIES.txt', 'w')
for t in incorrect_answers:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

##### 1.5.3.4 Comparison to wordlist

In [0]:
nltk_words = nltk.corpus.words.words('en')
tokens = set(huff_simp_bi_trigr10_vocab.token)
unusual_words = tokens.difference(nltk_words)
print("The number of unusual words:", len(unusual_words))

The number of unusual words: 40764


In [0]:
unusual_words

{'v_holder',
 'told_bloomberg',
 'luxury_brands',
 'universal_pictures',
 'cirque_du',
 'angers',
 'edits',
 'alcohol_intake',
 'rose_tico',
 'talib',
 'dorothy',
 'disrupts',
 'mc',
 'luxembourg',
 'feeders',
 'accolades',
 'ventured',
 'alarming_rate',
 'last_tycoon',
 'deutsch_wordpress',
 'mins',
 'became_apparent',
 'tamron_hall',
 'pendleton',
 'cortez_d_n',
 'phillippe',
 'sheila',
 'improvising',
 'pacific_press',
 'south_dakota',
 'crime_stoppers',
 'antibiotic_resistance',
 'ideagen',
 'tax_rate',
 'ailey',
 'tax_bracket',
 'water_bottles',
 'tried_to_convince',
 'smartphone_app',
 'ivanka_and_jared',
 'defunding_planned',
 'statue_of_robert',
 'recanted',
 'outings',
 'confederate_monuments',
 'juliana',
 'hate_crime',
 'groped',
 'abolitionists',
 'dan_scavino',
 'warwick',
 'editing',
 'new_testament',
 'armstrong',
 'esteban',
 'charlottesville_va',
 'cyber_criminals',
 'folic_acid',
 'walter_shaub',
 'tasers',
 'alexis_olympia',
 'widens',
 'professor_geoff',
 'somerhald

In [0]:
f = open('huff_simp_bi_trigr10_UNUSUAL.txt', 'w')
for t in unusual_words:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

## 2 Breitbart

### 2.1 Data exploration

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Data")
right_news = pd.read_csv("right_news_dates.csv")
breitbart = right_news[right_news.source_domain == "www.breitbart.com"]
del(right_news)
breitbart.head()

,date_publish,lang,source_domain,title,text
0,2019-07-30 11:42:14,en,www.breitbart.com,Boko Haram Kills Up to 70 Mourners at Funeral ...,The death toll from a suspected Boko Haram att...
1,2019-09-08 14:31:38,en,www.breitbart.com,I'll Help Johnson Get 100-Seat Majority in Ele...,Brexit Party leader Nigel Farage has said that...
2,2017-01-12 04:47:38,en,www.breitbart.com,ESPN: Chargers plan to announce move to LA,SIGN UP FOR OUR NEWSLETTER NEW YORK (AP) — The...
3,2019-07-18 10:54:40,en,www.breitbart.com,Iran Claims Revolutionary Guard Seize Foreign ...,Iran’s state TV claims marine elements of the ...
4,2019-07-03 21:35:37,en,www.breitbart.com,MTV to Air Raunchy Nicki Minaj Concert in Saud...,"DUBAI, United Arab Emirates (AP) — Saudi Arabi..."


In [0]:
print("Number of articles:", len(breitbart)) # 81018

Number of articles: 81018


#### 2.1.1 Select only articles published in or later than 2010

In [0]:
breitbart['year_publish'] = breitbart.apply(lambda row: int(row.date_publish[:4]) if (pd.notnull(row.date_publish)) else row.date_publish, axis=1)

In [0]:
print("The number of articles published in or later than 2010:",
      len(breitbart[breitbart.year_publish >= 2010]), ",",
      round(len(breitbart[breitbart.year_publish >= 2010])/len(breitbart[pd.notnull(breitbart.year_publish)])*100,2),
      "%")
print("The number of articles published earlier than 2010:",
      len(breitbart[breitbart.year_publish < 2010]), ",",
      round(len(breitbart[breitbart.year_publish < 2010])/len(breitbart[pd.notnull(breitbart.year_publish)])*100,2),
      "%")

The number of articles published in or later than 2010: 81016 , 100.0 %
The number of articles published earlier than 2010: 0 , 0.0 %


In [0]:
print("The number of articles with missing publish date:", breitbart.date_publish.isnull().sum())

The number of articles with missing publish date: 2


Articles with unknown publish date are kept due to the scarcity of the data. Basing on the rest of the data, I assume that majority of them were published later than 2010.

#### 2.1.2 Missing texts

In [0]:
print('Is there missing values?', breitbart.isnull().values.any())
print('Number of missing values:\n', breitbart.isnull().sum())
# Delete the rows with missing article text:
breitbart = breitbart.dropna(subset=['text'])

Is there missing values? True
Number of missing values:
 date_publish      2
lang              0
source_domain     0
title             0
text             34
year_publish      2
dtype: int64


In [0]:
print('Number of articles after filtering:', breitbart.shape[0]) # 80984

Number of articles after filtering: 80984


### 2.2 Pre-processing

#### 2.2.1 Include title in the text

In [0]:
breitbart['text_preprocessed'] = breitbart["title"].astype(str) + ". " + breitbart["text"]

#### 2.2.2 Remove links and nicknames

In [0]:
http_number, www_number, pic_number, at_number = 0, 0, 0, 0
for row in breitbart.text_preprocessed:
  if "http" in row:
    http_number += 1
  if 'www' in row:
    www_number += 1
  if 'pic.' in row:
    pic_number += 1
  if '@' in row:
    at_number += 1

print('The number of articles containing "http" in the text', http_number)
print('The number of articles containing "www" in the text', www_number)
print('The number of articles containing "pic." in the text', pic_number)
print('The number of articles containing "@" in the text', at_number)

The number of articles containing "http" in the text 11063
The number of articles containing "www" in the text 333
The number of articles containing "pic." in the text 10125
The number of articles containing "@" in the text 46296


In [0]:
breitbart['text_preprocessed'] = breitbart.apply(lambda row: re.sub(r'http\S+', '', row.text_preprocessed), axis=1) # including https
breitbart['text_preprocessed'] = breitbart.apply(lambda row: re.sub(r'www\.\S+', '', row.text_preprocessed), axis=1)
breitbart['text_preprocessed'] = breitbart.apply(lambda row: re.sub(r'pic\.\S+', '', row.text_preprocessed), axis=1)
breitbart['text_preprocessed'] = breitbart.apply(lambda row: re.sub(r'@\S+', '', row.text_preprocessed), axis=1)

#### 2.2.3 Gensim simple pre-processing

In [0]:
start_time = time.time()
breitbart['tokens'] = breitbart.apply(lambda row: gensim.utils.simple_preprocess(row.text_preprocessed, 
                                                                               deacc=False, min_len=1, max_len=28),
                                    axis=1)
end_time = time.time()
print("breitbart gensim simple_preprocess", round((end_time - start_time),2), "seconds")

breitbart gensim simple_preprocess 48.32 seconds


In [0]:
breitbart['words'] = breitbart.apply(lambda row: len(row.tokens), axis=1)

print("Total number of words for training:", sum(breitbart['words']))
print("Average number of words per article:", round(statistics.mean(breitbart['words']),0))

Total number of words for training: 39348959
Average number of words per article: 486.0


### 2.3 Word embeddings

#### 2.3.1 Bigrams generation

In [0]:
stopwords = stopwords.words("english")
print('I will use', len(stopwords), 'stop words from nltk')
common_terms = frozenset(stopwords)
stopwords

In [0]:
start_time = time.time()
# threshold for forming the phrases - higher means fewer phrases
# min_count (float, optional) – Ignore all words and bigrams with total collected count lower than this value
bigram_transformer = Phrases(breitbart.tokens,
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 10.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 57.06 seconds


In [0]:
start_time = time.time()
bigram_transformer_second = Phrases(bigram_transformer[breitbart.tokens],
                             min_count = 25, 
                             delimiter = b'_',
                             common_terms = common_terms,
                             scoring = 'default',
                             threshold = 30.0)
end_time = time.time()

print("Bigram transformer (Huffington Post) for", round((end_time - start_time),2), "seconds")

Bigram transformer (Huffington Post) for 173.76 seconds


Check the bigrams generated by the first round (on one article):

In [0]:
breitbart.tokens

0        [boko, haram, kills, up, to, mourners, at, fun...
1        [i, ll, help, johnson, get, seat, majority, in...
2        [espn, chargers, plan, to, announce, move, to,...
3        [iran, claims, revolutionary, guard, seize, fo...
4        [mtv, to, air, raunchy, nicki, minaj, concert,...
                               ...                        
81013    [there, will, be, revolution, blood, in, the, ...
81014    [slovenia, tightens, asylum, law, will, turn, ...
81015    [merkel, insists, i, m, fine, after, public, s...
81016    [nolte, associated, press, becomes, fake, news...
81017    [lion, ted, cruz, challenges, media, champions...
Name: tokens, Length: 80984, dtype: object

In [0]:
bigrams_stop_low_th = []

for token in bigram_transformer[breitbart.tokens[0]]:
  if "_" in token:
    bigrams_stop_low_th.append(token)
    
bigrams_stop_low_th

['boko_haram',
 'death_toll',
 'suspected_boko',
 'northeastern_nigeria',
 'borno_state',
 'body_count',
 'news_agency',
 'injuries_sustained',
 'state_run',
 'xinhua_news',
 'monday_morning',
 'al_jazeera',
 'heinous_attack',
 'boko_haram',
 'two_weeks',
 'two_weeks',
 'two_months',
 'president_muhammadu',
 'falsely_claimed',
 'boko_haram',
 'borno_state',
 'boko_haram',
 'wreaking_havoc',
 'islamic_state',
 'west_africa',
 'boko_haram',
 'allegedly_broke',
 'claimed_responsibility',
 'boko_haram',
 'al_jazeera',
 'boko_haram',
 'around_the_globe',
 'state_department',
 'boko_haram',
 'top_five',
 'deadliest_terrorist',
 'boko_haram',
 'killed_and_injured']

Check the bigrams generated by the second round (on one article):

In [0]:
bigrams_stop_low_th_second = []

for token in bigram_transformer_second[breitbart.tokens[0]]:
  if "_" in token:
    bigrams_stop_low_th_second.append(token)
    
bigrams_stop_low_th_second

['boko_haram',
 'death_toll',
 'suspected_boko',
 'northeastern_nigeria',
 'borno_state',
 'news_agency',
 'injuries_sustained',
 'state_run',
 'xinhua_news',
 'monday_morning',
 'al_jazeera',
 'heinous_attack',
 'boko_haram',
 'two_weeks',
 'two_weeks',
 'two_months',
 'falsely_claimed',
 'boko_haram',
 'borno_state',
 'boko_haram',
 'wreaking_havoc',
 'islamic_state',
 'west_africa',
 'boko_haram',
 'claimed_responsibility',
 'boko_haram',
 'al_jazeera',
 'boko_haram',
 'around_the_globe',
 'state_department',
 'boko_haram',
 'deadliest_terrorist',
 'boko_haram']

What is new after the second pass?

In [0]:
for t in bigrams_stop_low_th_second:
  if t not in bigrams_stop_low_th:
    print(t)

#### 2.3.2 Training

In [0]:
start_time = time.time()
breitbart10 = Word2Vec(bigram_transformer_second[breitbart['tokens']],
                      size = 300, window = 8, min_count = 25, workers = 4, sample = 0.00001,
                      sg = 1, compute_loss = True, callbacks = (), iter = 10)
end_time = time.time()

os.chdir("/content/gdrive/My Drive/CSS Project/Models")
breitbart10.save("breitbart10.model")
print("Word2Vec for breitbart for", round((end_time - start_time),2), "seconds")

Word2Vec for breitbart for 2044.3 seconds


#### 2.3.3 Analysis of quality

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Models")
breitbart10 = Word2Vec.load("breitbart10.model")

##### 2.3.3.1 Vocabulary

In [0]:
print("Vocabulary length:", len(breitbart10.wv.vocab), "words")

Vocabulary length: 44783 words


In [0]:
breitbart10_vocab = {}
for token in breitbart10.wv.vocab:
  breitbart10_vocab[token] = breitbart10.wv.vocab[token].count

breitbart10_vocab = pd.DataFrame.from_dict(breitbart10_vocab, orient='index', columns=['frequency'])
breitbart10_vocab['token'] = breitbart10_vocab.index
breitbart10_vocab = breitbart10_vocab.sort_values(by='frequency', ascending=False)
breitbart10_vocab = breitbart10_vocab.reset_index(inplace = False) 
breitbart10_vocab = breitbart10_vocab[['token', 'frequency']]

breitbart10_vocab.head()

,token,frequency
0,the,2227205
1,to,1102498
2,of,911652
3,and,861639
4,a,796713


In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")
breitbart10_vocab.to_csv(r'breitbart10_VOCAB.csv', index = False, header = True)

##### 2.3.3.2 Word semantic similarity

In [0]:
os.chdir("/content/gdrive/My Drive/CSS Project/Evaluation data")

In [0]:
wordsim353 = breitbart10.evaluate_word_pairs('wordsim353.tsv')
simverb3500 = breitbart10.evaluate_word_pairs('simverb-3500.txt')
men = breitbart10.evaluate_word_pairs('men.txt')
rw = breitbart10.evaluate_word_pairs('rw.txt')


print("Test dataset WordSim-353 Pearson:", round(wordsim353[0][0],2), ", Spearman:", round(wordsim353[1][0],2))
print("Test dataset SimVerb-3500 Pearson:", round(simverb3500[0][0],2), ", Spearman:", round(simverb3500[1][0],2))
print("Test dataset MEN Pearson:", round(men[0][0],2), ", Spearman:", round(men[1][0],2))
print("Test dataset RW Pearson:", round(rw[0][0],2), ", Spearman:", round(rw[1][0],2))

Test dataset WordSim-353 Pearson: 0.58 , Spearman: 0.59
Test dataset SimVerb-3500 Pearson: 0.16 , Spearman: 0.19
Test dataset MEN Pearson: 0.59 , Spearman: 0.59
Test dataset RW Pearson: 0.44 , Spearman: 0.49


##### 2.3.3.3 Word analogy 

In [0]:
analogy_google = breitbart10.wv.evaluate_word_analogies('questions-words.txt')
analogy_SemEval = breitbart10.wv.evaluate_word_analogies('semeval.txt')

print("Test dataset Google:", round(analogy_google[0],2))
print("Test dataset SemEval-2012:", round(analogy_SemEval[0],2))

Test dataset Google: 0.37
Test dataset SemEval-2012: 0.02


In [0]:
accuracy = breitbart10.wv.accuracy('questions-words.txt', case_insensitive=True)

incorrects = []
for section in range(len(accuracy)):
  for incorrect in accuracy[section]['incorrect']:
    incorrects.append(incorrect)
    
len(incorrects)

11728

In [0]:
incorrect_answers = []

for question in range(len(incorrects)):
  try:
    pos1 = incorrects[question][1].lower()
    pos2 = incorrects[question][2].lower()
    neg = incorrects[question][0].lower()
    answer = breitbart10.wv.most_similar(positive=[pos1,pos2], negative=[neg], topn=1)[0][0]
    incorrect_answers.append((neg,pos1,pos2,answer))
  except:
    continue
    
incorrect_answers

[('athens', 'greece', 'baghdad', 'krg'),
 ('athens', 'greece', 'hanoi', 'hanoi_vietnam'),
 ('athens', 'greece', 'kabul', 'afghan_government'),
 ('athens', 'greece', 'london', 'uk'),
 ('baghdad', 'iraq', 'kabul', 'taliban'),
 ('baghdad', 'iraq', 'london', 'mayor_sadiq'),
 ('beijing', 'china', 'hanoi', 'hanoi_vietnam'),
 ('beijing', 'china', 'helsinki', 'putin_in_helsinki'),
 ('beijing', 'china', 'kabul', 'taliban'),
 ('beijing', 'china', 'london', 'mayor_sadiq'),
 ('berlin', 'germany', 'hanoi', 'south_korea'),
 ('berlin', 'germany', 'kabul', 'afghan_government'),
 ('berlin', 'germany', 'london', 'uk'),
 ('cairo', 'egypt', 'hanoi', 'north_korea'),
 ('cairo', 'egypt', 'helsinki', 'putin_in_helsinki'),
 ('cairo', 'egypt', 'kabul', 'afghan_government'),
 ('cairo', 'egypt', 'london', 'mayor_sadiq'),
 ('canberra', 'australia', 'kabul', 'afghan_government'),
 ('canberra', 'australia', 'london', 'uk'),
 ('hanoi', 'vietnam', 'kabul', 'afghan_government'),
 ('hanoi', 'vietnam', 'london', 'mayor_s

In [0]:
f = open('breitbart10_ANALOGIES.txt', 'w')
for t in incorrect_answers:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()

##### 2.3.3.4 Comparison to wordlist

In [0]:
nltk_words = nltk.corpus.words.words('en')
tokens = set(breitbart10_vocab.token)
unusual_words = tokens.difference(nltk_words)
print("The number of unusual words:", len(unusual_words))

The number of unusual words: 30517


In [0]:
unusual_words

{'political_aisle',
 'plants',
 'help_fix',
 'weight_behind',
 'fellow_americans',
 'empowering',
 'rosneft',
 'federal_agents',
 'speeds',
 'kavanaugh',
 'aretha_franklin',
 'six_years',
 'ed_gillespie',
 'beneath_the_dignity',
 'plaguing',
 'alan_dershowitz',
 'laverne_cox',
 'violent_protests',
 'zimbabwean',
 'trumpers',
 'version_of_events',
 'budgeting',
 'claimed',
 'testified_under_oath',
 'zombies',
 'specially_designated',
 'departures',
 'asylum_reforms',
 'burst_into_flames',
 'public_service',
 'reinking',
 'tomi_lahren',
 'fifth_avenue',
 'investment_bank',
 'audiences',
 'felix',
 'linebacker',
 'politically_neutral',
 'arthur',
 'buttigieg',
 'sidelined_americans',
 'volunteers',
 'oakland_california',
 'showcasing',
 'homeland_defense',
 'bureaucracies',
 'dms',
 'able_to_obtain',
 'leavers',
 'strictest',
 'duarte',
 'asa',
 'åkesson',
 'tahrir',
 'bredesen',
 'small_percentage',
 'dress_code',
 'jane_robbins',
 'continue_to_grow',
 'quinney',
 'claimants',
 'cuban_co

In [0]:
f = open('breitbart10_UNUSUAL.txt', 'w')
for t in unusual_words:
  line = ' '.join(str(x) for x in t)
  f.write(line + '\n')
f.close()